In [35]:
!pip install sktime

In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = '/content/drive/MyDrive/데이콘 캐글 컴페티션/DACON_23.07_PowerConsumption/'
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [38]:
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
import sktime
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.utils.plotting import plot_series
import matplotlib.pyplot as pl24
from sklearn.metrics import make_scorer
from lightgbm import early_stopping
import xgboost as xgb
import seaborn as sns

In [39]:
def _smape(true, pred):
    true = np.array(true)
    pred = np.array(pred)
    v = 2 * np.abs(pred - true) / (np.abs(pred) + np.abs(true))
    output = np.mean(v) * 100
    return output
smape = make_scorer(_smape, greater_is_better=False)


def lightgbm_smape(y_true, y_pred):
    smape_val = _smape(y_true, y_pred)
    return 'SMAPE', smape_val, False

In [40]:
train = pd.read_csv(f"{DATA_PATH}train.csv")
test = pd.read_csv(f"{DATA_PATH}test.csv")
# building = pd.read_csv(f"{DATA_PATH}building_info.csv")
submission = pd.read_csv(f"{DATA_PATH}sample_submission.csv")

In [41]:

# train_ft = pd.read_csv(f'{DATA_PATH}train_ver3.csv')
# test_ft = pd.read_csv(f'{DATA_PATH}test_ver3.csv')

train_ft = pd.read_csv(f'{DATA_PATH}train_features_ys_rm.csv')
test_ft = pd.read_csv(f'{DATA_PATH}test_features_ys_rm.csv')


In [42]:
train_ft.shape, test_ft.shape

((201600, 34), (16800, 31))

In [43]:
train_ft.columns

Index(['power', 'building_num', 'date_time', 'temp', 'pred', 'wind', 'hum',
       'sunshine', 'solar_rad', 'type', 'time', 'day', 'month', 'week',
       'day_of_year', 'sin_time', 'cos_time', 'temp_RM', 'wind_RM', 'hum_RM',
       'temp_EMA', 'wind_EMA', 'hum_EMA', 'THI', 'feels_temp', 'CDH',
       'hour_mean', 'day_hour_mean', 'hour_std', 'heat_index', 'heat_wave',
       'temp_2', 'avg_power_change_rate_7d', 'avg_power_change_rate_24h'],
      dtype='object')

In [44]:
test_ft.columns

Index(['building_num', 'date_time', 'temp', 'pred', 'wind', 'hum', 'type',
       'time', 'day', 'month', 'week', 'day_of_year', 'sin_time', 'cos_time',
       'temp_RM', 'wind_RM', 'hum_RM', 'temp_EMA', 'wind_EMA', 'hum_EMA',
       'THI', 'feels_temp', 'CDH', 'hour_mean', 'day_hour_mean', 'hour_std',
       'heat_index', 'heat_wave', 'temp_2', 'avg_power_change_rate_7d',
       'avg_power_change_rate_24h'],
      dtype='object')

In [45]:
train_ft.columns

cols = ['energy_consum', 'building_num', 'date_time', 'temp', 'precip', 'wind', 'hum',
       'sunshine', 'solar_rad', 'building_type', 'time', 'day', 'month', 'week',
       'day_of_year', 'sin_time', 'cos_time', 'temp_RM', 'wind_RM', 'hum_RM',
       'temp_EMA', 'wind_EMA', 'hum_EMA', 'THI', 'feels_temp', 'CDH',
       'hour_mean', 'day_hour_mean', 'hour_std', 'heat_index', 'heat_wave',
        'temp_2', 'avg_power_change_rate_7d', 'avg_power_change_rate_24h']


train_ft.columns = cols

In [46]:
test_ft.columns

cols = ['building_num', 'date_time', 'temp', 'precip', 'wind', 'hum', 'building_type',
       'time', 'day', 'month', 'week', 'day_of_year', 'sin_time', 'cos_time',
       'temp_RM', 'wind_RM', 'hum_RM', 'temp_EMA', 'wind_EMA', 'hum_EMA',
       'THI', 'feels_temp', 'CDH', 'hour_mean', 'day_hour_mean', 'hour_std',
       'heat_index', 'heat_wave',
        'temp_2', 'avg_power_change_rate_7d', 'avg_power_change_rate_24h']

test_ft.columns = cols

In [47]:
train_ft.isna().sum().sum(), test_ft.isna().sum().sum()

(0, 0)

In [48]:
train_ft.shape, test_ft.shape

((201600, 34), (16800, 31))

In [49]:
cols = ['sunshine', 'solar_rad']
train_ft = train_ft.drop(columns=cols)

## 건물별 ft 생성

In [50]:
for i in range(1, 101):
    globals()[f'train_ft_{i}'] = train_ft[train_ft['building_num'] == i]

for i in range(1, 101):
    globals()[f'test_ft_{i}'] = test_ft[test_ft['building_num'] == i]

## 공휴일 반영

In [51]:
# 휴무일 지정(1: 휴무일, 0: 영업일)
train_ft_1['weekend'] = train_ft_1['day'].isin([6]) + 0
test_ft_1['weekend'] = test_ft_1['day'].isin([6]) + 0

mask = train_ft_1['day_of_year'] == 157
train_ft_1.loc[mask, 'weekend'] = 1

# 휴무일 지정(1: 휴무일, 0: 영업일)
train_ft_2['weekend'] = train_ft_2['day'].isin([0]) + 0
test_ft_2['weekend'] = test_ft_2['day'].isin([0]) + 0

# 휴무일 지정(1: 휴무일, 0: 영업일)
train_ft_3['weekend'] = train_ft_3['day'].isin([0]) + 0
test_ft_3['weekend'] = test_ft_3['day'].isin([0]) + 0

# 휴무일 지정(1: 휴무일, 0: 영업일)
train_ft_5['weekend'] = train_ft_5['day'].isin([0, 1, 2, 3]) + 0
test_ft_5['weekend'] = test_ft_5['day'].isin([0, 1, 2, 3]) + 0

# 휴무일 지정(1: 휴무일, 0: 영업일)
train_ft_6['weekend'] = 0
test_ft_6['weekend'] = 0

mask_train = (train_ft_6['day_of_year'] == 163) | (train_ft_6['day_of_year'] == 177)| (train_ft_6['day_of_year'] == 191)| (train_ft_6['day_of_year'] == 205)| (train_ft_6['day_of_year'] == 226)
train_ft_6.loc[mask_train, 'weekend'] = 1

mask_test = test_ft_6['day_of_year'] == 240
test_ft_6.loc[mask_test, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_7['weekend'] = train_ft_7['day'].isin([5, 6]) + 0
test_ft_7['weekend'] = test_ft_7['day'].isin([5, 6]) + 0

# 주말 지정(0: 주중, 1: 주말)
train_ft_8['weekend'] = train_ft_8['day'].isin([6]) + 0
test_ft_8['weekend'] = test_ft_8['day'].isin([6]) + 0

# 주말 지정(0: 주중, 1: 주말)
train_ft_9['weekend'] = 0
test_ft_9['weekend'] = 0

mask_train = (train_ft_9['day_of_year'] == 163) | (train_ft_9['day_of_year'] == 177)| (train_ft_9['day_of_year'] == 191)| (train_ft_9['day_of_year'] == 205)| (train_ft_9['day_of_year'] == 226)
train_ft_9.loc[mask_train, 'weekend'] = 1

mask_test = test_ft_9['day_of_year'] == 240
test_ft_9.loc[mask_test, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_12['weekend'] = train_ft_12['day'].isin([6]) + 0
test_ft_12['weekend'] = test_ft_12['day'].isin([6]) + 0

# 주말 지정(0: 주중, 1: 주말)
train_ft_14['weekend'] = train_ft_14['day'].isin([5,6]) + 0
test_ft_14['weekend'] = test_ft_14['day'].isin([5,6]) + 0

# 주말 지정(0: 주중, 1: 주말)
train_ft_15['weekend'] = train_ft_15['day'].isin([5,6]) + 0
test_ft_15['weekend'] = test_ft_15['day'].isin([5,6]) + 0

# 주말 지정(0: 주중, 1: 주말)
train_ft_16['weekend'] = train_ft_16['day'].isin([5, 6]) + 0
test_ft_16['weekend'] = test_ft_16['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_16['day_of_year'] == 157) | (train_ft_16['day_of_year'] == 227)
train_ft_16.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_17['weekend'] = train_ft_17['day'].isin([5, 6]) + 0
test_ft_17['weekend'] = test_ft_17['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_17['day_of_year'] == 157) | (train_ft_17['day_of_year'] == 227)
train_ft_17.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_18['weekend'] = train_ft_18['day'].isin([5, 6]) + 0
test_ft_18['weekend'] = test_ft_18['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_18['day_of_year'] == 157) | (train_ft_18['day_of_year'] == 227)
train_ft_18.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_19['weekend'] = train_ft_19['day'].isin([5, 6]) + 0
test_ft_19['weekend'] = test_ft_19['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_19['day_of_year'] == 157) | (train_ft_19['day_of_year'] == 227)
train_ft_19.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_20['weekend'] = train_ft_20['day'].isin([5, 6]) + 0

test_ft_20['weekend'] = test_ft_20['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_20['day_of_year'] == 157) | (train_ft_20['day_of_year'] == 227)
train_ft_20.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_21['weekend'] = train_ft_21['day'].isin([5, 6]) + 0
test_ft_21['weekend'] = test_ft_21['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_21['day_of_year'] == 157) | (train_ft_21['day_of_year'] == 227)
train_ft_21.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_22['weekend'] = train_ft_22['day'].isin([5, 6]) + 0
test_ft_22['weekend'] = test_ft_22['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_22['day_of_year'] == 157) | (train_ft_22['day_of_year'] == 227)
train_ft_22.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_23['weekend'] = train_ft_23['day'].isin([5, 6]) + 0
test_ft_23['weekend'] = test_ft_23['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_23['day_of_year'] == 157) | (train_ft_23['day_of_year'] == 227)
train_ft_23.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_24['weekend'] = train_ft_24['day'].isin([5, 6]) + 0
test_ft_24['weekend'] = test_ft_24['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_24['day_of_year'] == 157) | (train_ft_24['day_of_year'] == 227)
train_ft_24.loc[mask, 'weekend'] = 1

# 휴무일 지정(0: 영업일, 1: 휴무일)
train_ft_25['weekend'] = (train_ft_25['day'] == 0) + 0
test_ft_25['weekend'] = (test_ft_25['day'] == 0) + 0

# 8월 16일, 8월 17일 휴무일로 예상.
mask = (train_ft_25['day_of_year'] == 228) | (train_ft_25['day_of_year'] == 229)
train_ft_25.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_26['weekend'] = train_ft_26['day'].isin([5, 6]) + 0
test_ft_26['weekend'] = test_ft_26['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_26['day_of_year'] == 157) | (train_ft_26['day_of_year'] == 227)
train_ft_26.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_27['weekend'] = train_ft_27['day'].isin([5, 6]) + 0
test_ft_27['weekend'] = test_ft_27['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_27['day_of_year'] == 157) | (train_ft_27['day_of_year'] == 227)
train_ft_27.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_28['weekend'] = train_ft_28['day'].isin([5, 6]) + 0
test_ft_28['weekend'] = test_ft_28['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_28['day_of_year'] == 157) | (train_ft_28['day_of_year'] == 227)
train_ft_28.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_29['weekend'] = train_ft_29['day'].isin([5, 6]) + 0
test_ft_29['weekend'] = test_ft_29['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_29['day_of_year'] == 157) | (train_ft_29['day_of_year'] == 227)
train_ft_29.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_30['weekend'] = train_ft_30['day'].isin([5, 6]) + 0
test_ft_30['weekend'] = test_ft_30['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_30['day_of_year'] == 157) | (train_ft_30['day_of_year'] == 227)
train_ft_30.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_31['weekend'] = train_ft_31['day'].isin([5, 6]) + 0
test_ft_31['weekend'] = test_ft_31['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_31['day_of_year'] == 157) | (train_ft_31['day_of_year'] == 227)
train_ft_31.loc[mask, 'weekend'] = 1



<ipython-input-51-731d943b1092>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_ft_1['weekend'] = train_ft_1['day'].isin([6]) + 0
<ipython-input-51-731d943b1092>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_ft_1['weekend'] = test_ft_1['day'].isin([6]) + 0
<ipython-input-51-731d943b1092>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

In [52]:
# 휴무일 지정(1: 휴무일, 0: 영업일)
train_ft_37['closed'] = 0
mask = (train_ft_37['day_of_year'] == 171) | (train_ft_37['day_of_year'] == 192) | (train_ft_37['day_of_year'] == 220)
train_ft_37.loc[mask, 'closed'] = 1

test_ft_37['closed'] = 0

# 휴무일 지정(1: 휴무일, 0: 영업일)
train_ft_38['closed'] = 0
mask = (train_ft_38['day_of_year'] == 164) | (train_ft_38['day_of_year'] == 206) | (train_ft_38['day_of_year'] == 213)
train_ft_38.loc[mask, 'closed'] = 1

test_ft_38['closed'] = 0

# 휴무일 지정(1: 휴무일, 0: 영업일)
train_ft_39['closed'] = 0
mask = (train_ft_39['day_of_year'] == 199) | (train_ft_39['day_of_year'] == 220)
train_ft_39.loc[mask, 'closed'] = 1

test_ft_39['closed'] = 0

# 휴무일 지정(1: 휴무일, 0: 영업일)
train_ft_40['closed'] = 0
mask = (train_ft_40['day_of_year'] == 171) | (train_ft_40['day_of_year'] == 199) | (train_ft_40['day_of_year'] == 220)
train_ft_40.loc[mask, 'closed'] = 1

test_ft_40['closed'] = 0

# 휴무일 지정(1: 휴무일, 0: 영업일)
train_ft_41['closed'] = 0
mask = (train_ft_41['day_of_year'] == 178) | (train_ft_41['day_of_year'] == 206) | (train_ft_41['day_of_year'] == 220)
train_ft_41.loc[mask, 'closed'] = 1

test_ft_41['closed'] = 0

# 주말 지정(0: 주중, 1: 주말)
train_ft_41['weekend'] = train_ft_41['day'].isin([5, 6]) + 0

test_ft_41['weekend'] = test_ft_41['day'].isin([5, 6]) + 0

# 휴무일 지정(1: 휴무일, 0: 영업일)
train_ft_42['closed'] = 0
mask = (train_ft_42['day_of_year'] == 164) | (train_ft_42['day_of_year'] == 192) | (train_ft_42['day_of_year'] == 234)
train_ft_42.loc[mask, 'closed'] = 1

test_ft_42['closed'] = 0

# 주말 지정(0: 주중, 1: 주말)
train_ft_43['weekend'] = train_ft_43['day'].isin([5, 6]) + 0

test_ft_43['weekend'] = test_ft_43['day'].isin([5, 6]) + 0

# 주말 지정(0: 주중, 1: 주말)
train_ft_44['weekend'] = train_ft_44['day'].isin([5, 6]) + 0

test_ft_44['weekend'] = test_ft_44['day'].isin([5, 6]) + 0

# 주말 지정(0: 주중, 1: 주말)
train_ft_45['weekend'] = train_ft_45['day'].isin([5, 6]) + 0

test_ft_45['weekend'] = test_ft_45['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_45['day_of_year'] == 157) | (train_ft_45['day_of_year'] == 227)
train_ft_45.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_46['weekend'] = train_ft_46['day'].isin([5, 6]) + 0

test_ft_46['weekend'] = test_ft_46['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_46['day_of_year'] == 157) | (train_ft_46['day_of_year'] == 227)
train_ft_46.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_47['weekend'] = train_ft_47['day'].isin([5, 6]) + 0

test_ft_47['weekend'] = test_ft_47['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_47['day_of_year'] == 157) | (train_ft_47['day_of_year'] == 227)
train_ft_47.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_48['weekend'] = train_ft_48['day'].isin([5, 6]) + 0

test_ft_48['weekend'] = test_ft_48['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_48['day_of_year'] == 157) | (train_ft_48['day_of_year'] == 227)
train_ft_48.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_49['weekend'] = train_ft_49['day'].isin([5, 6]) + 0

test_ft_49['weekend'] = test_ft_49['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_49['day_of_year'] == 157) | (train_ft_49['day_of_year'] == 227)
train_ft_49.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_50['weekend'] = train_ft_50['day'].isin([5, 6]) + 0

test_ft_50['weekend'] = test_ft_50['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_50['day_of_year'] == 157) | (train_ft_50['day_of_year'] == 227)
train_ft_50.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_51['weekend'] = train_ft_51['day'].isin([5, 6]) + 0

test_ft_51['weekend'] = test_ft_51['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_51['day_of_year'] == 157) | (train_ft_51['day_of_year'] == 227)
train_ft_51.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_52['weekend'] = train_ft_52['day'].isin([5, 6]) + 0

test_ft_52['weekend'] = test_ft_52['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_52['day_of_year'] == 157) | (train_ft_52['day_of_year'] == 227)
train_ft_52.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_53['weekend'] = train_ft_53['day'].isin([5, 6]) + 0

test_ft_53['weekend'] = test_ft_53['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_53['day_of_year'] == 157) | (train_ft_53['day_of_year'] == 227)
train_ft_53.loc[mask, 'weekend'] = 1

# 휴무일 지정(0: 영업일, 1: 휴무일)
train_ft_54['weekend'] = (train_ft_54['day'] == 0) + 0

test_ft_54['weekend'] = (test_ft_54['day'] == 0) + 0

# 8월 16일, 8월 17일 휴무일로 예상.
mask = (train_ft_54['day_of_year'] == 228) | (train_ft_54['day_of_year'] == 229)
train_ft_54.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_55['weekend'] = train_ft_55['day'].isin([5, 6]) + 0

test_ft_55['weekend'] = test_ft_55['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_55['day_of_year'] == 157) | (train_ft_55['day_of_year'] == 227)
train_ft_55.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_56['weekend'] = train_ft_56['day'].isin([5, 6]) + 0

test_ft_56['weekend'] = test_ft_56['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_56['day_of_year'] == 157) | (train_ft_56['day_of_year'] == 227)
train_ft_56.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_57['weekend'] = train_ft_57['day'].isin([5, 6]) + 0

test_ft_57['weekend'] = test_ft_57['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_57['day_of_year'] == 157) | (train_ft_57['day_of_year'] == 227)
train_ft_57.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_58['weekend'] = train_ft_58['day'].isin([5, 6]) + 0

test_ft_58['weekend'] = test_ft_58['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_58['day_of_year'] == 157) | (train_ft_58['day_of_year'] == 227)
train_ft_58.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_59['weekend'] = train_ft_59['day'].isin([5, 6]) + 0

test_ft_59['weekend'] = test_ft_59['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_59['day_of_year'] == 157) | (train_ft_59['day_of_year'] == 227)
train_ft_59.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_60['weekend'] = train_ft_60['day'].isin([5, 6]) + 0

test_ft_60['weekend'] = test_ft_60['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_60['day_of_year'] == 157) | (train_ft_60['day_of_year'] == 227)
train_ft_60.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_61['weekend'] = train_ft_61['day'].isin([5, 6]) + 0

test_ft_61['weekend'] = test_ft_61['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_61['day_of_year'] == 157) | (train_ft_61['day_of_year'] == 227)
train_ft_61.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_62['weekend'] = train_ft_62['day'].isin([5, 6]) + 0

test_ft_62['weekend'] = test_ft_62['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_62['day_of_year'] == 157) | (train_ft_62['day_of_year'] == 227)
train_ft_62.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_63['weekend'] = train_ft_63['day'].isin([5, 6]) + 0

test_ft_63['weekend'] = test_ft_63['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_63['day_of_year'] == 157) | (train_ft_63['day_of_year'] == 227)
train_ft_63.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_64['weekend'] = train_ft_64['day'].isin([5, 6]) + 0

test_ft_64['weekend'] = test_ft_64['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_64['day_of_year'] == 157) | (train_ft_64['day_of_year'] == 227)
train_ft_64.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_65['weekend'] = train_ft_65['day'].isin([5, 6]) + 0

test_ft_65['weekend'] = test_ft_65['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_65['day_of_year'] == 157) | (train_ft_65['day_of_year'] == 227)
train_ft_65.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_66['weekend'] = train_ft_66['day'].isin([5, 6]) + 0

test_ft_66['weekend'] = test_ft_66['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_66['day_of_year'] == 157) | (train_ft_66['day_of_year'] == 227)
train_ft_66.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_67['weekend'] = train_ft_67['day'].isin([5, 6]) + 0

test_ft_67['weekend'] = test_ft_67['day'].isin([5, 6]) + 0

# 주말 지정(0: 주중, 1: 주말)
train_ft_67['weekend'] = train_ft_67['day'].isin([5, 6]) + 0

test_ft_67['weekend'] = test_ft_67['day'].isin([5, 6]) + 0

# 주말 지정(0: 주중, 1: 주말)
train_ft_68['weekend'] = train_ft_68['day'].isin([5, 6]) + 0

test_ft_68['weekend'] = test_ft_68['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_68['day_of_year'] == 157) | (train_ft_68['day_of_year'] == 227)
train_ft_68.loc[mask, 'weekend'] = 1




<ipython-input-52-fd22beefbdde>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_ft_37['closed'] = 0
<ipython-input-52-fd22beefbdde>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_ft_37['closed'] = 0
<ipython-input-52-fd22beefbdde>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

In [53]:
# 주말 지정(0: 주중, 1: 주말)
train_ft_69['weekend'] = train_ft_69['day'].isin([5, 6]) + 0
test_ft_69['weekend'] = test_ft_69['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_69['day_of_year'] == 157) | (train_ft_69['day_of_year'] == 227)
train_ft_69.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_70['weekend'] = train_ft_70['day'].isin([5, 6]) + 0
test_ft_70['weekend'] = test_ft_70['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_70['day_of_year'] == 157) | (train_ft_70['day_of_year'] == 227)
train_ft_70.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_71['weekend'] = train_ft_71['day'].isin([5, 6]) + 0
test_ft_71['weekend'] = test_ft_71['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_71['day_of_year'] == 157) | (train_ft_71['day_of_year'] == 227)
train_ft_71.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_72['weekend'] = train_ft_72['day'].isin([5, 6]) + 0
test_ft_72['weekend'] = test_ft_72['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_72['day_of_year'] == 157) | (train_ft_72['day_of_year'] == 227)
train_ft_72.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_73['weekend'] = train_ft_73['day'].isin([5, 6]) + 0
test_ft_73['weekend'] = test_ft_73['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_73['day_of_year'] == 157) | (train_ft_73['day_of_year'] == 227)
train_ft_73.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_74['weekend'] = train_ft_74['day'].isin([5, 6]) + 0
test_ft_74['weekend'] = test_ft_74['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_74['day_of_year'] == 157) | (train_ft_74['day_of_year'] == 227)
train_ft_74.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_75['weekend'] = train_ft_75['day'].isin([5, 6, 0]) + 0
test_ft_75['weekend'] = test_ft_75['day'].isin([5, 6, 0]) + 0

# 주말 지정(0: 주중, 1: 주말)
train_ft_76['weekend'] = train_ft_76['day'].isin([5, 6]) + 0
test_ft_76['weekend'] = test_ft_76['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_76['day_of_year'] == 157) | (train_ft_76['day_of_year'] == 227)
train_ft_76.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_77['weekend'] = train_ft_77['day'].isin([5, 6]) + 0
test_ft_77['weekend'] = test_ft_77['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_77['day_of_year'] == 157) | (train_ft_77['day_of_year'] == 227)
train_ft_77.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_78['weekend'] = train_ft_78['day'].isin([5, 6]) + 0

test_ft_78['weekend'] = test_ft_78['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_78['day_of_year'] == 157) | (train_ft_78['day_of_year'] == 227)
train_ft_78.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_79['weekend'] = train_ft_79['day'].isin([5, 6]) + 0

test_ft_79['weekend'] = test_ft_79['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_79['day_of_year'] == 157) | (train_ft_79['day_of_year'] == 227)
train_ft_79.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_80['weekend'] = train_ft_80['day'].isin([5, 6]) + 0

test_ft_80['weekend'] = test_ft_80['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_80['day_of_year'] == 157) | (train_ft_80['day_of_year'] == 227)
train_ft_80.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_82['weekend'] = train_ft_82['day'].isin([5, 6]) + 0
test_ft_82['weekend'] = test_ft_82['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_82['day_of_year'] == 157) | (train_ft_82['day_of_year'] == 227)
train_ft_82.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_83['weekend'] = train_ft_83['day'].isin([5, 6]) + 0

test_ft_83['weekend'] = test_ft_83['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_83['day_of_year'] == 157) | (train_ft_83['day_of_year'] == 227)
train_ft_83.loc[mask, 'weekend'] = 1

# 주말 지정(0: 주중, 1: 주말)
train_ft_84['weekend'] = train_ft_84['day'].isin([5, 6]) + 0
test_ft_84['weekend'] = test_ft_84['day'].isin([5, 6]) + 0

# 공휴일 주말로 지정(공휴일이 2일 밖에 존재하지 않으며, 주말과 차이가 없다고 판단)
mask = (train_ft_84['day_of_year'] == 157) | (train_ft_84['day_of_year'] == 227)
train_ft_84.loc[mask, 'weekend'] = 1

# 휴무일 지정(1: 휴무일, 0: 영업일)
train_ft_86['weekend'] = 0
test_ft_86['weekend'] = 0

mask_train = (train_ft_86['day_of_year'] == 161) | (train_ft_86['day_of_year'] == 177)| (train_ft_86['day_of_year'] == 191)| (train_ft_86['day_of_year'] == 205)| (train_ft_86['day_of_year'] == 222)
train_ft_86.loc[mask_train, 'weekend'] = 1

# 휴무일 지정(1: 휴무일, 0: 영업일)
train_ft_87['weekend'] = 0
test_ft_87['weekend'] = 0

mask_train = (train_ft_87['day_of_year'] == 163) | (train_ft_87['day_of_year'] == 177)| (train_ft_87['day_of_year'] == 191)| (train_ft_87['day_of_year'] == 205)| (train_ft_87['day_of_year'] == 226)
train_ft_87.loc[mask_train, 'weekend'] = 1

mask_test = test_ft_87['day_of_year'] == 240
test_ft_87.loc[mask_test, 'weekend'] = 1

# 휴무일 지정(1: 휴무일, 0: 영업일)
train_ft_88['weekend'] = 0
test_ft_88['weekend'] = 0

mask_train = (train_ft_88['day_of_year'] == 163) | (train_ft_88['day_of_year'] == 177)| (train_ft_88['day_of_year'] == 191)| (train_ft_88['day_of_year'] == 205)| (train_ft_88['day_of_year'] == 226)
train_ft_88.loc[mask_train, 'weekend'] = 1

mask_test = test_ft_88['day_of_year'] == 240
test_ft_88.loc[mask_test, 'weekend'] = 1

# 휴무일 지정(1: 휴무일, 0: 영업일)
train_ft_89['weekend'] = 0
test_ft_89['weekend'] = 0

mask_train = (train_ft_89['day_of_year'] == 163) | (train_ft_89['day_of_year'] == 177)| (train_ft_89['day_of_year'] == 191)| (train_ft_89['day_of_year'] == 205)| (train_ft_89['day_of_year'] == 226)
train_ft_89.loc[mask_train, 'weekend'] = 1

mask_test = test_ft_89['day_of_year'] == 240
test_ft_89.loc[mask_test, 'weekend'] = 1

# 휴무일 지정(1: 휴무일, 0: 영업일)
train_ft_90['weekend'] = 0
test_ft_90['weekend'] = 0

mask_train = (train_ft_90['day_of_year'] == 163) | (train_ft_90['day_of_year'] == 177)| (train_ft_90['day_of_year'] == 191)| (train_ft_90['day_of_year'] == 205)| (train_ft_90['day_of_year'] == 226)
train_ft_90.loc[mask_train, 'weekend'] = 1

mask_test = test_ft_90['day_of_year'] == 240
test_ft_90.loc[mask_test, 'weekend'] = 1

# 휴무일 지정(1: 휴무일, 0: 영업일)
train_ft_91['weekend'] = 0
test_ft_91['weekend'] = 0

mask_train = (train_ft_91['day_of_year'] == 163) | (train_ft_91['day_of_year'] == 177)| (train_ft_91['day_of_year'] == 191)| (train_ft_91['day_of_year'] == 205)| (train_ft_91['day_of_year'] == 226)
train_ft_91.loc[mask_train, 'weekend'] = 1

mask_test = test_ft_91['day_of_year'] == 240
test_ft_91.loc[mask_test, 'weekend'] = 1

# 휴무일 지정(1: 휴무일, 0: 영업일)
train_ft_92['weekend'] = 0
test_ft_92['weekend'] = 0

mask_train = (train_ft_92['day_of_year'] == 163) | (train_ft_92['day_of_year'] == 177)| (train_ft_92['day_of_year'] == 191)| (train_ft_92['day_of_year'] == 205)| (train_ft_92['day_of_year'] == 226)
train_ft_92.loc[mask_train, 'weekend'] = 1

mask_test = test_ft_92['day_of_year'] == 240
test_ft_92.loc[mask_test, 'weekend'] = 1




<ipython-input-53-3dcc50ac59b7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_ft_69['weekend'] = train_ft_69['day'].isin([5, 6]) + 0
<ipython-input-53-3dcc50ac59b7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_ft_69['weekend'] = test_ft_69['day'].isin([5, 6]) + 0
<ipython-input-53-3dcc50ac59b7>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

- 공휴일 활용 피처 : weekend가 있는 건물들에 대해서 weekend의 왜도

## 개별 피처 추가 및 제거

In [54]:


columns_to_drop = [ 'temp','wind', 'hum', # RM 때문에 제거할 칼럼
                   'heat_wave', 'heat_index',  # 필요 없어서 제거할 칼럼
                   'day_of_year','building_num','date_time','building_type','CDH','hour_std','precip','hour_mean'] #일조의 변화율?
                    # 'precip', 'hour_mean']  # 기본 제거할 칼럼

for i in range(1, 101):
    globals()[f'train_ft_{i}'] = globals()[f'train_ft_{i}'].drop(columns=columns_to_drop)
    globals()[f'test_ft_{i}'] = globals()[f'test_ft_{i}'].drop(columns=columns_to_drop)

- 함수

In [55]:
train_ft.iloc[:, 2:]

,date_time,temp,precip,wind,hum,building_type,time,day,month,week,...,feels_temp,CDH,hour_mean,day_hour_mean,hour_std,heat_index,heat_wave,temp_2,avg_power_change_rate_7d,avg_power_change_rate_24h
0,20220602 00,18.1,0.0,0.4,67.0,건물기타,0,3,6,22,...,28.818026,-1.850000,1713.711429,1627.80,444.306331,0.0,0,327.61,0.069171,0.021710
1,20220602 01,17.6,0.0,0.8,69.0,건물기타,1,3,6,22,...,27.987268,-4.591667,1629.468571,1550.08,437.718795,0.0,0,309.76,0.072765,0.023154
2,20220602 02,17.2,0.0,0.7,71.0,건물기타,2,3,6,22,...,27.080175,-8.266667,1513.305714,1431.12,410.362488,0.0,0,295.84,0.071155,0.025346
3,20220602 03,17.2,0.0,2.2,75.0,건물기타,3,3,6,22,...,26.152269,-12.866667,1443.122857,1372.20,389.916262,0.0,0,295.84,0.067707,0.030905
4,20220602 04,17.2,0.0,2.0,81.0,건물기타,4,3,6,22,...,25.248724,-18.333333,1452.808571,1381.72,379.995696,0.0,0,295.84,0.063264,0.033128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201595,20220824 19,23.1,0.0,0.9,86.0,호텔및리조트,19,2,8,34,...,30.510262,-35.641667,1012.602857,976.06,161.150392,0.0,0,533.61,0.039282,0.004240
201596,20220824 20,22.4,0.0,1.3,86.0,호텔및리조트,20,2,8,34,...,30.441848,-33.391667,930.205714,892.92,137.040302,0.0,0,501.76,0.040570,0.004927
201597,20220824 21,21.3,0.0,1.0,92.0,호텔및리조트,21,2,8,34,...,30.233938,-31.266667,831.714286,786.62,128.125250,0.0,0,453.69,0.053521,0.006987
201598,20220824 22,21.0,0.0,0.3,94.0,호텔및리조트,22,2,8,34,...,29.960151,-29.483333,724.480000,667.94,112.413470,0.0,0,441.00,0.036985,0.008615


In [56]:
!pip install CatBoost

In [57]:
from catboost import CatBoostRegressor, Pool
import numpy as np
from tqdm import tqdm

# 데이터셋 만들기(마지막 7일간의 데이터를 검증데이터셋으로 사용)
def make_dataset(train_df, test_df):
    train_x = train_df.iloc[:, 2:]
    train_y = train_df['energy_consum']

    test_x = test_df.iloc[:, 1:]
    return train_x, train_y, test_x

def n_fold_train_test_split(train_x, train_y, n, test_size=168, week=12):
    x_valid = train_x[test_size * (week-n-1):test_size * (week-n)]
    y_valid = train_y[test_size * (week-n-1):test_size * (week-n)]
    x_train = train_x.drop(index=list(x_valid.index))
    y_train = train_y.drop(index=list(y_valid.index))
    return x_train, y_train, x_valid, y_valid

# CatBoostRegressor 모델
def catboost_predict(model, x_train, y_train, x_valid, y_valid, stopping=300):
    train_pool = Pool(data=x_train, label=y_train)
    valid_pool = Pool(data=x_valid, label=y_valid)

    model.fit(train_pool, eval_set=valid_pool, early_stopping_rounds=stopping, use_best_model=True, verbose=100)

    pred = model.predict(valid_pool)
    return pred

# test dataset 예측
def catboost_test_predict(model, train_x, train_y, test_x):
    train_pool = Pool(data=train_x, label=train_y)
    model.fit(train_pool)
    catboost_pred = model.predict(test_x)
    return catboost_pred

# _smape 함수 정의
def _smape(true, pred):
    true = np.array(true)
    pred = np.array(pred)
    v = 2 * np.abs(pred - true) / (np.abs(pred) + np.abs(true))
    output = np.mean(v) * 100
    return output

# CatBoost 모델 사용
catboost_preds = np.array([])
fold = 4
smape_score_mean = []

def catboost_evaluation(train_ft, test_ft):
    global catboost_preds

    # 모델 생성 및 custom_metric 설정
    model = CatBoostRegressor(random_seed=SEED, custom_metric=['SMAPE'])

    train_x, train_y, test_x = make_dataset(train_ft, test_ft)

    # 건물별 fold별 smape score
    smape_score = []
    for n in tqdm(range(fold)):
        x_train, y_train, x_valid, y_valid = n_fold_train_test_split(train_x, train_y, n=n)

        pred = catboost_predict(model, x_train, y_train, x_valid, y_valid, stopping=120)
        score = _smape(y_valid, pred)
        smape_score.append(score)

    smape_score_mean.append(np.mean(smape_score))

    catboost_pred = catboost_test_predict(model, train_x, train_y, test_x)
    catboost_preds = np.concatenate([catboost_preds, catboost_pred])

# # 사용 예시
# train_ft_1 = ...  # 훈련 데이터 프레임
# test_ft_1 = ...   # 테스트 데이터 프레임
# SEED = 42         # 랜덤 시드

# catboost_evaluation(train_ft_1, test_ft_1)

# print("Mean SMAPE Scores:", smape_score_mean)


In [58]:
# from catboost import CatBoostRegressor, Pool

# # 데이터셋 만들기(마지막 7일간의 데이터를 검증데이터셋으로 사용)
# def make_dataset(train_df, test_df):
#     train_x = train_df.iloc[:, 2:]
#     train_y = train_df['energy_consum']

#     test_x = test_df.iloc[:, 1:]
#     return train_x, train_y, test_x

# def n_fold_train_test_split(train_x, train_y, n, test_size=168, week=11):
#     x_valid = train_x[test_size * (week-n-1):test_size * (week-n)]
#     y_valid = train_y[test_size * (week-n-1):test_size * (week-n)]
#     x_train = train_x.drop(index=list(x_valid.index))
#     y_train = train_y.drop(index=list(y_valid.index))
#     return x_train, y_train, x_valid, y_valid

# # CatBoostRegressor 모델
# def catboost_predict(model, x_train, y_train, x_valid, y_valid, stopping=300):
#     train_pool = Pool(data=x_train, label=y_train)
#     valid_pool = Pool(data=x_valid, label=y_valid)

#     model.fit(train_pool, eval_set=valid_pool, early_stopping_rounds=stopping, use_best_model=True, verbose=100)

#     pred = model.predict(valid_pool)
#     return pred

# # test dataset 예측
# def catboost_test_predict(model, train_x, train_y, test_x):
#     train_pool = Pool(data=train_x, label=train_y)
#     model.fit(train_pool)
#     catboost_pred = model.predict(test_x)
#     return catboost_pred

# # CatBoost 모델 사용

# # CatBoost 모델 사용
# catboost_preds = np.array([])
# fold = 4
# smape_score_mean = []

# def catboost_evaluation(train_ft, test_ft): #catboost이나 이름만!!!!!!!

#     global catboost_preds

#     model = CatBoostRegressor(random_seed=SEED, loss_function='MAPE')
#     train_x, train_y, test_x = make_dataset(train_ft, test_ft)

#     # 건물별 fold별 smape score
#     smape_score = []
#     for n in tqdm(range(fold)):
#         x_train, y_train, x_valid, y_valid = n_fold_train_test_split(train_x, train_y, n=n)

#         pred = catboost_predict(model, x_train, y_train, x_valid, y_valid, stopping=120)
#         score = _smape(y_valid, pred)
#         smape_score.append(score)

#     smape_score_mean.append(np.mean(smape_score))

#     catboost_pred = catboost_test_predict(model, train_x, train_y, test_x)
#     catboost_preds = np.concatenate([catboost_preds, catboost_pred])

# # print(smape_score_mean)
# #

## 1

In [59]:
catboost_evaluation(train_ft_1, test_ft_1) #함수 내용은 catboost임

  0%|          | 0/4 [00:00<?, ?it/s]

Learning rate set to 0.056075
0:	learn: 1070.6639139	test: 987.3768138	best: 987.3768138 (0)	total: 2.87ms	remaining: 2.87s
100:	learn: 169.6602457	test: 189.4218738	best: 189.4218738 (100)	total: 260ms	remaining: 2.32s
200:	learn: 135.1038492	test: 175.7208564	best: 175.7208564 (200)	total: 704ms	remaining: 2.8s
300:	learn: 113.2974770	test: 170.1345818	best: 170.1345818 (300)	total: 1.28s	remaining: 2.97s
400:	learn: 98.5684092	test: 167.8944798	best: 167.7495790 (383)	total: 1.97s	remaining: 2.95s
500:	learn: 86.7041639	test: 167.9918084	best: 167.7038007 (419)	total: 2.63s	remaining: 2.62s
600:	learn: 77.6907902	test: 166.3812176	best: 166.3536395 (599)	total: 3.3s	remaining: 2.19s
700:	learn: 69.9608641	test: 165.6438041	best: 165.6086437 (697)	total: 3.97s	remaining: 1.69s
800:	learn: 63.5373822	test: 165.0377341	best: 164.9644064 (799)	total: 4.43s	remaining: 1.1s
900:	learn: 57.9983082	test: 164.8271615	best: 164.6565620 (842)	total: 4.68s	remaining: 515ms


 25%|██▌       | 1/4 [00:05<00:15,  5.10s/it]

Stopped by overfitting detector  (120 iterations wait)

bestTest = 164.656562
bestIteration = 842

Shrink model to first 843 iterations.
Learning rate set to 0.056075
0:	learn: 1072.9210630	test: 960.7318481	best: 960.7318481 (0)	total: 4.56ms	remaining: 4.55s
100:	learn: 171.2339516	test: 179.8672426	best: 176.9408159 (96)	total: 289ms	remaining: 2.57s
200:	learn: 134.0567766	test: 178.6511011	best: 174.3598322 (148)	total: 535ms	remaining: 2.13s
Stopped by overfitting detector  (120 iterations wait)

bestTest = 174.3598322
bestIteration = 148

Shrink model to first 149 iterations.


 50%|█████     | 2/4 [00:05<00:05,  2.57s/it]

...

 75%|███████▌  | 3/4 [00:06<00:01,  1.83s/it]

300:	learn: 109.0067021	test: 314.2840905	best: 312.5845936 (234)	total: 768ms	remaining: 1.78s
Stopped by overfitting detector  (120 iterations wait)

bestTest = 312.5845936
bestIteration = 234

Shrink model to first 235 iterations.
Learning rate set to 0.056075
0:	learn: 1064.1576897	test: 1066.4482293	best: 1066.4482293 (0)	total: 2.68ms	remaining: 2.68s
100:	learn: 164.3225411	test: 258.2008239	best: 258.0717347 (92)	total: 259ms	remaining: 2.3s


100%|██████████| 4/4 [00:07<00:00,  1.87s/it]

200:	learn: 131.2699083	test: 264.5636599	best: 257.9818174 (102)	total: 516ms	remaining: 2.05s
Stopped by overfitting detector  (120 iterations wait)

bestTest = 257.9818174
bestIteration = 102

Shrink model to first 103 iterations.
Learning rate set to 0.045739
0:	learn: 1073.2608128	total: 4.09ms	remaining: 4.08s
1:	learn: 1030.5287857	total: 7.16ms	remaining: 3.57s
2:	learn: 990.7916750	total: 9.78ms	remaining: 3.25s
3:	learn: 951.1782067	total: 12.2ms	remaining: 3.05s
4:	learn: 914.1110335	total: 14.7ms	remaining: 2.92s
5:	learn: 880.2953196	total: 16.4ms	remaining: 2.72s
6:	learn: 846.8971450	total: 19ms	remaining: 2.69s
7:	learn: 814.7064177	total: 21.5ms	remaining: 2.67s
8:	learn: 784.7921323	total: 24.2ms	remaining: 2.67s
9:	learn: 757.4618102	total: 27.1ms	remaining: 2.69s
10:	learn: 731.4697955	total: 30ms	remaining: 2.69s
11:	learn: 704.3182340	total: 32.8ms	remaining: 2.7s
12:	learn: 677.5139807	total: 35.5ms	remaining: 2.7s
13:	learn: 652.6500498	total: 38.1ms	remaining: 

33:	learn: 355.5267858	total: 93.6ms	remaining: 2.66s
34:	learn: 346.7287963	total: 96.2ms	remaining: 2.65s
35:	learn: 338.5794076	total: 99.1ms	remaining: 2.65s
36:	learn: 330.6690467	total: 102ms	remaining: 2.65s
37:	learn: 323.0456740	total: 105ms	remaining: 2.67s
38:	learn: 316.6716798	total: 108ms	remaining: 2.67s
39:	learn: 309.9874108	total: 111ms	remaining: 2.67s
40:	learn: 303.8560084	total: 114ms	remaining: 2.67s
41:	learn: 297.5309329	total: 117ms	remaining: 2.67s
42:	learn: 292.1637616	total: 120ms	remaining: 2.67s
43:	learn: 286.8085270	total: 123ms	remaining: 2.66s
44:	learn: 282.0775608	total: 125ms	remaining: 2.66s
45:	learn: 276.8933236	total: 128ms	remaining: 2.66s
46:	learn: 272.3321016	total: 131ms	remaining: 2.65s
47:	learn: 267.8672715	total: 134ms	remaining: 2.65s
48:	learn: 263.3759031	total: 136ms	remaining: 2.65s
49:	learn: 259.3060912	total: 139ms	remaining: 2.64s
50:	learn: 255.5054981	total: 142ms	remaining: 2.64s
51:	learn: 251.9841569	total: 144ms	remaini

In [60]:
train_ft_1.columns

Index(['energy_consum', 'time', 'day', 'month', 'week', 'sin_time', 'cos_time',
       'temp_RM', 'wind_RM', 'hum_RM', 'temp_EMA', 'wind_EMA', 'hum_EMA',
       'THI', 'feels_temp', 'day_hour_mean', 'temp_2',
       'avg_power_change_rate_7d', 'avg_power_change_rate_24h', 'weekend'],
      dtype='object')

## 2

In [61]:
catboost_evaluation(train_ft_2, test_ft_2)

  0%|          | 0/4 [00:00<?, ?it/s]

Learning rate set to 0.056075
0:	learn: 591.7665151	test: 501.9240244	best: 501.9240244 (0)	total: 4.18ms	remaining: 4.17s
100:	learn: 145.7118135	test: 224.1752479	best: 222.6242278 (88)	total: 558ms	remaining: 4.96s
200:	learn: 116.5294711	test: 204.0925107	best: 203.4268017 (195)	total: 814ms	remaining: 3.24s
300:	learn: 96.5757712	test: 204.0586703	best: 202.1688332 (270)	total: 1.08s	remaining: 2.51s


 25%|██▌       | 1/4 [00:01<00:04,  1.38s/it]

Stopped by overfitting detector  (120 iterations wait)

bestTest = 202.1688332
bestIteration = 270

Shrink model to first 271 iterations.
Learning rate set to 0.056075
0:	learn: 580.0031392	test: 645.0939633	best: 645.0939633 (0)	total: 2.64ms	remaining: 2.64s
100:	learn: 144.7381775	test: 216.3981300	best: 215.8232869 (98)	total: 290ms	remaining: 2.58s
200:	learn: 115.1302638	test: 214.4060457	best: 214.2773592 (177)	total: 702ms	remaining: 2.79s
300:	learn: 96.9774990	test: 211.7879661	best: 210.3277900 (242)	total: 950ms	remaining: 2.21s
Stopped by overfitting detector  (120 iterations wait)

bestTest = 210.32779
bestIteration = 242

Shrink model to first 243 iterations.


 50%|█████     | 2/4 [00:02<00:02,  1.28s/it]

...

 75%|███████▌  | 3/4 [00:07<00:02,  2.99s/it]

999:	learn: 42.9207701	test: 174.2972558	best: 174.0903430 (963)	total: 4.9s	remaining: 0us

bestTest = 174.090343
bestIteration = 963

Shrink model to first 964 iterations.
Learning rate set to 0.056075
0:	learn: 585.6002637	test: 584.0166669	best: 584.0166669 (0)	total: 3.64ms	remaining: 3.64s
100:	learn: 145.7672003	test: 220.6977774	best: 208.3081281 (52)	total: 580ms	remaining: 5.16s


100%|██████████| 4/4 [00:08<00:00,  2.13s/it]

Stopped by overfitting detector  (120 iterations wait)

bestTest = 208.3081281
bestIteration = 52

Shrink model to first 53 iterations.
Learning rate set to 0.045739
0:	learn: 590.5781618	total: 3.02ms	remaining: 3.02s
1:	learn: 570.1814955	total: 6.09ms	remaining: 3.04s
2:	learn: 550.7745740	total: 8.99ms	remaining: 2.99s
3:	learn: 532.6341909	total: 11.7ms	remaining: 2.92s
4:	learn: 514.5608379	total: 14.7ms	remaining: 2.92s
5:	learn: 496.9205582	total: 17.5ms	remaining: 2.9s
6:	learn: 479.8227300	total: 20.4ms	remaining: 2.89s
7:	learn: 464.4614723	total: 23.2ms	remaining: 2.87s
8:	learn: 448.7843544	total: 25.9ms	remaining: 2.85s
9:	learn: 434.0665519	total: 28.8ms	remaining: 2.85s
10:	learn: 420.3940076	total: 31.7ms	remaining: 2.85s
11:	learn: 407.5108320	total: 34.5ms	remaining: 2.84s
12:	learn: 395.4340263	total: 37.3ms	remaining: 2.83s
13:	learn: 383.3792578	total: 40ms	remaining: 2.81s
14:	learn: 371.3224713	total: 42.7ms	remaining: 2.8s
15:	learn: 360.4110262	total: 45.6ms	r

52:	learn: 192.9976007	total: 159ms	remaining: 2.85s
53:	learn: 191.3295148	total: 164ms	remaining: 2.87s
54:	learn: 190.0668778	total: 167ms	remaining: 2.87s
55:	learn: 188.4516635	total: 171ms	remaining: 2.88s
56:	learn: 187.0806684	total: 175ms	remaining: 2.89s
57:	learn: 185.5181469	total: 181ms	remaining: 2.95s
58:	learn: 184.3771416	total: 184ms	remaining: 2.94s
59:	learn: 183.3914065	total: 187ms	remaining: 2.93s
60:	learn: 181.9614813	total: 190ms	remaining: 2.92s
61:	learn: 180.4167547	total: 193ms	remaining: 2.92s
62:	learn: 179.4082634	total: 196ms	remaining: 2.92s
63:	learn: 178.4997639	total: 199ms	remaining: 2.91s
64:	learn: 177.5023518	total: 202ms	remaining: 2.9s
65:	learn: 176.4763777	total: 204ms	remaining: 2.89s
66:	learn: 175.6046010	total: 207ms	remaining: 2.88s
67:	learn: 174.8146965	total: 210ms	remaining: 2.88s
68:	learn: 173.6847486	total: 213ms	remaining: 2.87s
69:	learn: 172.8282320	total: 216ms	remaining: 2.87s
70:	learn: 172.2177505	total: 219ms	remaining: 

## 3

In [62]:
catboost_evaluation(train_ft_3, test_ft_3)

  0%|          | 0/4 [00:00<?, ?it/s]

Learning rate set to 0.056075
0:	learn: 610.9731634	test: 515.1598985	best: 515.1598985 (0)	total: 6.05ms	remaining: 6.04s
100:	learn: 181.2663698	test: 314.9032399	best: 254.7666215 (33)	total: 725ms	remaining: 6.46s


 25%|██▌       | 1/4 [00:01<00:03,  1.14s/it]

Stopped by overfitting detector  (120 iterations wait)

bestTest = 254.7666215
bestIteration = 33

Shrink model to first 34 iterations.
Learning rate set to 0.056075
0:	learn: 598.8366899	test: 663.5827306	best: 663.5827306 (0)	total: 7.26ms	remaining: 7.26s
100:	learn: 175.4018124	test: 329.9178802	best: 328.6116574 (93)	total: 591ms	remaining: 5.26s
200:	learn: 130.2829452	test: 304.4281899	best: 304.4281899 (200)	total: 1.33s	remaining: 5.3s
300:	learn: 102.4133297	test: 292.3888157	best: 291.4684741 (297)	total: 2.02s	remaining: 4.7s
400:	learn: 86.9604797	test: 285.1776173	best: 284.9754751 (380)	total: 2.72s	remaining: 4.06s
500:	learn: 73.8819116	test: 279.0839679	best: 279.0839679 (500)	total: 3.42s	remaining: 3.4s
600:	learn: 65.2404371	test: 274.9655470	best: 274.8308180 (596)	total: 3.96s	remaining: 2.63s
700:	learn: 58.0353375	test: 273.9621764	best: 273.9621764 (700)	total: 4.44s	remaining: 1.9s
800:	learn: 52.5738747	test: 272.7868731	best: 272.6316962 (781)	total: 5.51s	

 50%|█████     | 2/4 [00:09<00:10,  5.10s/it]

...

 75%|███████▌  | 3/4 [00:11<00:03,  3.83s/it]

Stopped by overfitting detector  (120 iterations wait)

bestTest = 213.1374149
bestIteration = 141

Shrink model to first 142 iterations.
Learning rate set to 0.056075
0:	learn: 607.5673733	test: 558.8685757	best: 558.8685757 (0)	total: 5.34ms	remaining: 5.33s
100:	learn: 183.7617095	test: 263.2629322	best: 262.5734287 (88)	total: 688ms	remaining: 6.13s
200:	learn: 134.6481429	test: 255.7465520	best: 254.3445122 (164)	total: 1.32s	remaining: 5.27s
300:	learn: 106.2861948	test: 254.8286228	best: 253.8966613 (257)	total: 1.88s	remaining: 4.36s
400:	learn: 89.2093903	test: 254.6169084	best: 253.3443457 (309)	total: 2.56s	remaining: 3.83s
500:	learn: 77.4154244	test: 252.3296552	best: 252.2995564 (492)	total: 3.13s	remaining: 3.12s
600:	learn: 67.9345853	test: 252.8999556	best: 251.5408701 (543)	total: 3.68s	remaining: 2.44s


100%|██████████| 4/4 [00:15<00:00,  3.88s/it]

Stopped by overfitting detector  (120 iterations wait)

bestTest = 251.5408701
bestIteration = 543

Shrink model to first 544 iterations.
Learning rate set to 0.045739
0:	learn: 608.2493683	total: 11.7ms	remaining: 11.7s
1:	learn: 590.4025474	total: 25.1ms	remaining: 12.5s
2:	learn: 572.2514839	total: 31.6ms	remaining: 10.5s
3:	learn: 555.5838818	total: 45.7ms	remaining: 11.4s
4:	learn: 539.6446119	total: 52.1ms	remaining: 10.4s
5:	learn: 524.0045200	total: 56.8ms	remaining: 9.41s
6:	learn: 510.0573237	total: 61.3ms	remaining: 8.69s
7:	learn: 496.2712989	total: 64.1ms	remaining: 7.95s
8:	learn: 483.0672099	total: 67.4ms	remaining: 7.42s
9:	learn: 470.8353143	total: 71ms	remaining: 7.03s
10:	learn: 458.8138495	total: 73.7ms	remaining: 6.62s
11:	learn: 448.8169008	total: 79.9ms	remaining: 6.58s
12:	learn: 439.1550445	total: 82.9ms	remaining: 6.29s
13:	learn: 429.9090022	total: 86.3ms	remaining: 6.08s
14:	learn: 421.1267719	total: 89.6ms	remaining: 5.88s
15:	learn: 411.1325225	total: 92.2

28:	learn: 323.7655323	total: 145ms	remaining: 4.87s
29:	learn: 317.8923137	total: 154ms	remaining: 4.96s
30:	learn: 314.3167793	total: 172ms	remaining: 5.37s
31:	learn: 309.2517484	total: 179ms	remaining: 5.4s
32:	learn: 304.9932519	total: 185ms	remaining: 5.43s
33:	learn: 300.6561304	total: 192ms	remaining: 5.47s
34:	learn: 296.3865818	total: 199ms	remaining: 5.48s
35:	learn: 293.2862098	total: 205ms	remaining: 5.49s
36:	learn: 290.4464575	total: 213ms	remaining: 5.54s
37:	learn: 285.9050598	total: 221ms	remaining: 5.6s
38:	learn: 283.4654845	total: 228ms	remaining: 5.62s
39:	learn: 280.8697419	total: 236ms	remaining: 5.67s
40:	learn: 278.1895790	total: 244ms	remaining: 5.71s
41:	learn: 274.8275525	total: 252ms	remaining: 5.76s
42:	learn: 271.4310581	total: 260ms	remaining: 5.79s
43:	learn: 268.0884495	total: 268ms	remaining: 5.83s
44:	learn: 265.9998732	total: 279ms	remaining: 5.93s
45:	learn: 264.4429846	total: 286ms	remaining: 5.92s
46:	learn: 262.3380011	total: 303ms	remaining: 6

## 4

In [63]:
catboost_evaluation(train_ft_4, test_ft_4)

  0%|          | 0/4 [00:00<?, ?it/s]

Learning rate set to 0.056075
0:	learn: 272.8092677	test: 299.5004220	best: 299.5004220 (0)	total: 2.61ms	remaining: 2.6s
100:	learn: 42.6423542	test: 63.1171121	best: 57.8987343 (50)	total: 273ms	remaining: 2.43s


 25%|██▌       | 1/4 [00:00<00:01,  1.79it/s]

Stopped by overfitting detector  (120 iterations wait)

bestTest = 57.89873425
bestIteration = 50

Shrink model to first 51 iterations.
Learning rate set to 0.056075
0:	learn: 271.2436135	test: 317.5538113	best: 317.5538113 (0)	total: 4.81ms	remaining: 4.8s
100:	learn: 43.1364560	test: 69.3890424	best: 69.3890424 (100)	total: 638ms	remaining: 5.68s
200:	learn: 36.2857332	test: 63.8896130	best: 63.7971527 (195)	total: 1.24s	remaining: 4.93s
300:	learn: 31.0997248	test: 63.8451882	best: 63.3172402 (254)	total: 1.69s	remaining: 3.92s
400:	learn: 27.4373053	test: 63.2350219	best: 62.9921000 (361)	total: 2.23s	remaining: 3.34s
500:	learn: 24.4472246	test: 62.3434064	best: 62.3228565 (499)	total: 2.84s	remaining: 2.83s
600:	learn: 21.9684668	test: 62.1751283	best: 62.0772249 (582)	total: 3.38s	remaining: 2.25s


 50%|█████     | 2/4 [00:04<00:05,  2.68s/it]

...

 75%|███████▌  | 3/4 [00:05<00:01,  1.85s/it]

200:	learn: 35.6563025	test: 62.3468987	best: 61.6269790 (121)	total: 698ms	remaining: 2.77s
Stopped by overfitting detector  (120 iterations wait)

bestTest = 61.62697896
bestIteration = 121

Shrink model to first 122 iterations.
Learning rate set to 0.056075
0:	learn: 268.3074909	test: 345.0397368	best: 345.0397368 (0)	total: 2.81ms	remaining: 2.81s
100:	learn: 42.6513095	test: 72.7121799	best: 72.7095600 (99)	total: 255ms	remaining: 2.27s
200:	learn: 35.6805287	test: 71.1598195	best: 71.1598195 (200)	total: 511ms	remaining: 2.03s
300:	learn: 30.2618766	test: 70.1195995	best: 70.0339561 (255)	total: 771ms	remaining: 1.79s
400:	learn: 26.5759627	test: 69.4190832	best: 69.4190832 (400)	total: 1.03s	remaining: 1.54s
500:	learn: 23.6861819	test: 68.9912667	best: 68.8759739 (490)	total: 1.3s	remaining: 1.29s
600:	learn: 21.3389159	test: 68.8642393	best: 68.7934661 (594)	total: 1.55s	remaining: 1.03s
700:	learn: 19.2602500	test: 68.4170989	best: 68.4170989 (700)	total: 1.82s	remaining: 778

100%|██████████| 4/4 [00:08<00:00,  2.07s/it]

999:	learn: 14.6848063	test: 67.5301974	best: 67.5301974 (999)	total: 2.57s	remaining: 0us

bestTest = 67.5301974
bestIteration = 999

Learning rate set to 0.045739
0:	learn: 277.5861386	total: 2.75ms	remaining: 2.75s
1:	learn: 266.7238805	total: 5.4ms	remaining: 2.69s
2:	learn: 256.0272294	total: 8.32ms	remaining: 2.76s
3:	learn: 245.5831816	total: 11.2ms	remaining: 2.78s
4:	learn: 236.1976622	total: 13.9ms	remaining: 2.77s
5:	learn: 226.9726643	total: 16.6ms	remaining: 2.75s
6:	learn: 218.0690265	total: 19.3ms	remaining: 2.73s
7:	learn: 209.5692229	total: 22.1ms	remaining: 2.74s
8:	learn: 201.6687543	total: 24.9ms	remaining: 2.74s
9:	learn: 194.1284834	total: 27.7ms	remaining: 2.74s
10:	learn: 187.0937587	total: 30.6ms	remaining: 2.75s
11:	learn: 180.2398332	total: 33.3ms	remaining: 2.74s
12:	learn: 174.0160800	total: 36.1ms	remaining: 2.74s
13:	learn: 167.7936416	total: 38.9ms	remaining: 2.74s
14:	learn: 161.6807603	total: 41.7ms	remaining: 2.74s
15:	learn: 156.0429484	total: 44.5ms

48:	learn: 64.9839016	total: 149ms	remaining: 2.89s
49:	learn: 64.0377921	total: 152ms	remaining: 2.88s
50:	learn: 63.1506892	total: 155ms	remaining: 2.88s
51:	learn: 62.2155396	total: 157ms	remaining: 2.87s
52:	learn: 61.3392550	total: 160ms	remaining: 2.86s
53:	learn: 60.4779215	total: 163ms	remaining: 2.85s
54:	learn: 59.6284902	total: 166ms	remaining: 2.85s
55:	learn: 58.9609512	total: 171ms	remaining: 2.88s
56:	learn: 58.2922433	total: 174ms	remaining: 2.88s
57:	learn: 57.7144028	total: 178ms	remaining: 2.89s
58:	learn: 57.1276376	total: 181ms	remaining: 2.89s
59:	learn: 56.6165531	total: 184ms	remaining: 2.88s
60:	learn: 56.0816109	total: 186ms	remaining: 2.87s
61:	learn: 55.5904846	total: 189ms	remaining: 2.86s
62:	learn: 55.0824226	total: 192ms	remaining: 2.85s
63:	learn: 54.6232058	total: 195ms	remaining: 2.85s
64:	learn: 54.1905814	total: 197ms	remaining: 2.84s
65:	learn: 53.6782734	total: 200ms	remaining: 2.83s
66:	learn: 53.2547983	total: 203ms	remaining: 2.83s
67:	learn: 5

## 5

In [64]:
catboost_evaluation(train_ft_5, test_ft_5)

  0%|          | 0/4 [00:00<?, ?it/s]

Learning rate set to 0.056075
0:	learn: 1624.9010368	test: 1836.2165280	best: 1836.2165280 (0)	total: 3.08ms	remaining: 3.07s
100:	learn: 260.3674447	test: 260.1129322	best: 260.1129322 (100)	total: 283ms	remaining: 2.52s


 25%|██▌       | 1/4 [00:00<00:01,  1.57it/s]

200:	learn: 177.6929969	test: 310.7032178	best: 260.0639177 (101)	total: 533ms	remaining: 2.12s
Stopped by overfitting detector  (120 iterations wait)

bestTest = 260.0639177
bestIteration = 101

Shrink model to first 102 iterations.
Learning rate set to 0.056075
0:	learn: 1630.1119440	test: 1783.1455508	best: 1783.1455508 (0)	total: 2.65ms	remaining: 2.64s
100:	learn: 248.3573764	test: 300.7049228	best: 300.7049228 (100)	total: 265ms	remaining: 2.36s
200:	learn: 169.6859392	test: 288.6820909	best: 288.5471406 (179)	total: 554ms	remaining: 2.2s
300:	learn: 131.1065596	test: 287.1549960	best: 285.4629622 (267)	total: 808ms	remaining: 1.88s


 50%|█████     | 2/4 [00:01<00:01,  1.08it/s]

...

 75%|███████▌  | 3/4 [00:03<00:01,  1.11s/it]

Learning rate set to 0.056075
0:	learn: 1623.4757654	test: 1851.7341375	best: 1851.7341375 (0)	total: 5.74ms	remaining: 5.74s
100:	learn: 248.2246103	test: 332.7182647	best: 332.7182647 (100)	total: 263ms	remaining: 2.34s
200:	learn: 172.9214632	test: 287.2376176	best: 287.2376176 (200)	total: 511ms	remaining: 2.03s
300:	learn: 135.8414934	test: 287.0211636	best: 285.8313262 (284)	total: 1.15s	remaining: 2.66s
400:	learn: 112.1500608	test: 286.8248646	best: 284.0258131 (319)	total: 1.76s	remaining: 2.64s


100%|██████████| 4/4 [00:05<00:00,  1.30s/it]

Stopped by overfitting detector  (120 iterations wait)

bestTest = 284.0258131
bestIteration = 319

Shrink model to first 320 iterations.
Learning rate set to 0.045739
0:	learn: 1656.3154076	total: 6.13ms	remaining: 6.12s
1:	learn: 1589.7113430	total: 12.4ms	remaining: 6.17s
2:	learn: 1532.2870929	total: 20.4ms	remaining: 6.79s
3:	learn: 1475.4583679	total: 28.2ms	remaining: 7.02s
4:	learn: 1422.1928166	total: 36.2ms	remaining: 7.2s
5:	learn: 1369.4681315	total: 44.4ms	remaining: 7.36s
6:	learn: 1320.2349200	total: 50.5ms	remaining: 7.16s
7:	learn: 1273.9316360	total: 58.3ms	remaining: 7.23s
8:	learn: 1233.0119721	total: 67.5ms	remaining: 7.43s
9:	learn: 1187.0201424	total: 74.1ms	remaining: 7.34s
10:	learn: 1146.0069123	total: 80.7ms	remaining: 7.25s
11:	learn: 1107.4914098	total: 86.6ms	remaining: 7.13s
12:	learn: 1067.7553220	total: 91.9ms	remaining: 6.98s
13:	learn: 1031.3815471	total: 100ms	remaining: 7.05s
14:	learn: 994.6689962	total: 109ms	remaining: 7.14s
15:	learn: 962.193332

19:	learn: 841.8413196	total: 145ms	remaining: 7.08s
20:	learn: 814.2445688	total: 153ms	remaining: 7.14s
21:	learn: 789.7726647	total: 159ms	remaining: 7.06s
22:	learn: 763.6558602	total: 168ms	remaining: 7.13s
23:	learn: 739.2896289	total: 177ms	remaining: 7.21s
24:	learn: 716.0759023	total: 183ms	remaining: 7.14s
25:	learn: 696.1403841	total: 191ms	remaining: 7.15s
26:	learn: 678.0985214	total: 198ms	remaining: 7.14s
27:	learn: 656.1047372	total: 206ms	remaining: 7.15s
28:	learn: 637.2498006	total: 213ms	remaining: 7.14s
29:	learn: 619.8702510	total: 220ms	remaining: 7.13s
30:	learn: 601.4578241	total: 228ms	remaining: 7.12s
31:	learn: 584.7268942	total: 235ms	remaining: 7.12s
32:	learn: 572.0394155	total: 243ms	remaining: 7.12s
33:	learn: 559.1580333	total: 250ms	remaining: 7.11s
34:	learn: 545.7148843	total: 258ms	remaining: 7.11s
35:	learn: 532.9697083	total: 265ms	remaining: 7.1s
36:	learn: 521.5752425	total: 272ms	remaining: 7.09s
37:	learn: 511.7441122	total: 280ms	remaining: 

## 6

In [65]:
catboost_evaluation(train_ft_6, test_ft_6)

  0%|          | 0/4 [00:00<?, ?it/s]

Learning rate set to 0.056075
0:	learn: 735.7212900	test: 750.7038931	best: 750.7038931 (0)	total: 2.78ms	remaining: 2.78s
100:	learn: 75.9510859	test: 82.4305863	best: 82.3987550 (99)	total: 582ms	remaining: 5.18s
200:	learn: 58.3469980	test: 78.8148167	best: 78.6811362 (198)	total: 1.06s	remaining: 4.2s


 25%|██▌       | 1/4 [00:01<00:05,  1.79s/it]

300:	learn: 48.7688723	test: 80.3654823	best: 78.6811362 (198)	total: 1.58s	remaining: 3.68s
Stopped by overfitting detector  (120 iterations wait)

bestTest = 78.6811362
bestIteration = 198

Shrink model to first 199 iterations.
Learning rate set to 0.056075
0:	learn: 737.4457791	test: 732.3887661	best: 732.3887661 (0)	total: 6.68ms	remaining: 6.67s
100:	learn: 75.9770518	test: 72.3508925	best: 71.2057512 (77)	total: 532ms	remaining: 4.74s


 50%|█████     | 2/4 [00:03<00:02,  1.46s/it]

Stopped by overfitting detector  (120 iterations wait)

bestTest = 71.20575119
bestIteration = 77

Shrink model to first 78 iterations.
Learning rate set to 0.056075
0:	learn: 731.1094377	test: 800.8419678	best: 800.8419678 (0)	total: 4.37ms	remaining: 4.37s
100:	learn: 77.2073186	test: 88.2888374	best: 88.2888374 (100)	total: 562ms	remaining: 5s
200:	learn: 59.4536060	test: 74.0814121	best: 74.0548565 (199)	total: 1.34s	remaining: 5.34s
300:	learn: 48.9249101	test: 70.9550015	best: 70.9452221 (299)	total: 2.04s	remaining: 4.75s
400:	learn: 41.7859012	test: 70.3916402	best: 70.3417328 (398)	total: 2.92s	remaining: 4.36s


 75%|███████▌  | 3/4 [00:06<00:02,  2.44s/it]

500:	learn: 36.6530662	test: 70.5558439	best: 70.2045324 (415)	total: 3.38s	remaining: 3.36s
Stopped by overfitting detector  (120 iterations wait)

bestTest = 70.20453238
bestIteration = 415

Shrink model to first 416 iterations.
Learning rate set to 0.056075
0:	learn: 731.2508263	test: 800.1344990	best: 800.1344990 (0)	total: 2.71ms	remaining: 2.71s
100:	learn: 73.3865426	test: 130.5808474	best: 130.0727739 (81)	total: 254ms	remaining: 2.26s
200:	learn: 57.0134203	test: 131.0108972	best: 129.4844754 (132)	total: 499ms	remaining: 1.98s
Stopped by overfitting detector  (120 iterations wait)

bestTest = 129.4844754
bestIteration = 132

Shrink model to first 133 iterations.


100%|██████████| 4/4 [00:07<00:00,  1.84s/it]


Learning rate set to 0.045739
0:	learn: 743.6046183	total: 8.02ms	remaining: 8.01s
1:	learn: 712.5413856	total: 14.3ms	remaining: 7.11s
2:	learn: 683.4135851	total: 22.3ms	remaining: 7.41s
3:	learn: 655.9533240	total: 30.9ms	remaining: 7.7s
4:	learn: 629.6498074	total: 37ms	remaining: 7.36s
5:	learn: 604.1973395	total: 40.9ms	remaining: 6.77s
6:	learn: 579.8229822	total: 48.2ms	remaining: 6.84s
7:	learn: 557.2612038	total: 53.9ms	remaining: 6.68s
8:	learn: 535.2418110	total: 61.3ms	remaining: 6.75s
9:	learn: 514.9041273	total: 69.1ms	remaining: 6.84s
10:	learn: 494.1186818	total: 76.4ms	remaining: 6.87s
11:	learn: 475.4214963	total: 85.3ms	remaining: 7.02s
12:	learn: 458.0245571	total: 91.1ms	remaining: 6.92s
13:	learn: 440.3677739	total: 100ms	remaining: 7.05s
14:	learn: 423.2087322	total: 108ms	remaining: 7.09s
15:	learn: 407.1662714	total: 115ms	remaining: 7.1s
16:	learn: 392.3792066	total: 123ms	remaining: 7.12s
17:	learn: 377.4405337	total: 131ms	remaining: 7.16s
18:	learn: 363.53

## 7

In [66]:
catboost_evaluation(train_ft_7, test_ft_7)

  0%|          | 0/4 [00:00<?, ?it/s]

Learning rate set to 0.056075
0:	learn: 278.2807952	test: 289.9938891	best: 289.9938891 (0)	total: 2.98ms	remaining: 2.97s
100:	learn: 61.5481320	test: 75.4414127	best: 74.9201145 (88)	total: 262ms	remaining: 2.33s


 25%|██▌       | 1/4 [00:00<00:02,  1.46it/s]

200:	learn: 46.5406722	test: 76.6523776	best: 74.5969822 (125)	total: 511ms	remaining: 2.03s
Stopped by overfitting detector  (120 iterations wait)

bestTest = 74.59698219
bestIteration = 125

Shrink model to first 126 iterations.
Learning rate set to 0.056075
0:	learn: 278.8980986	test: 283.2866891	best: 283.2866891 (0)	total: 2.81ms	remaining: 2.8s
100:	learn: 60.4074442	test: 88.9587256	best: 87.5242223 (59)	total: 272ms	remaining: 2.42s


 50%|█████     | 2/4 [00:01<00:01,  1.68it/s]

...

 75%|███████▌  | 3/4 [00:03<00:01,  1.17s/it]

Learning rate set to 0.056075
0:	learn: 277.2838327	test: 302.6316614	best: 302.6316614 (0)	total: 2.63ms	remaining: 2.63s
100:	learn: 60.8986196	test: 89.4728480	best: 85.9425085 (59)	total: 272ms	remaining: 2.42s
200:	learn: 46.6112017	test: 85.2192126	best: 84.6324327 (192)	total: 516ms	remaining: 2.05s


100%|██████████| 4/4 [00:03<00:00,  1.01it/s]

300:	learn: 38.4456961	test: 85.1935805	best: 84.6324327 (192)	total: 784ms	remaining: 1.82s
Stopped by overfitting detector  (120 iterations wait)

bestTest = 84.63243271
bestIteration = 192

Shrink model to first 193 iterations.
Learning rate set to 0.045739
0:	learn: 281.1250337	total: 2.75ms	remaining: 2.75s
1:	learn: 270.2987738	total: 5.62ms	remaining: 2.8s
2:	learn: 260.1644380	total: 8.48ms	remaining: 2.82s
3:	learn: 250.5992312	total: 11.2ms	remaining: 2.8s
4:	learn: 241.5741641	total: 14.1ms	remaining: 2.81s
5:	learn: 232.7009578	total: 16.8ms	remaining: 2.78s
6:	learn: 224.9398126	total: 19.6ms	remaining: 2.77s
7:	learn: 217.0906031	total: 22.4ms	remaining: 2.77s
8:	learn: 209.6730950	total: 25.1ms	remaining: 2.77s
9:	learn: 203.0165637	total: 27.7ms	remaining: 2.75s
10:	learn: 195.9269726	total: 30.6ms	remaining: 2.75s
11:	learn: 189.4294723	total: 33.4ms	remaining: 2.75s
12:	learn: 183.7747977	total: 36.3ms	remaining: 2.76s
13:	learn: 177.4918599	total: 39.1ms	remaining: 2

44:	learn: 90.7289744	total: 127ms	remaining: 2.69s
45:	learn: 89.8121367	total: 129ms	remaining: 2.68s
46:	learn: 88.7699399	total: 132ms	remaining: 2.68s
47:	learn: 87.8628812	total: 135ms	remaining: 2.67s
48:	learn: 86.9874356	total: 138ms	remaining: 2.67s
49:	learn: 86.1085635	total: 142ms	remaining: 2.7s
50:	learn: 85.1230313	total: 147ms	remaining: 2.73s
51:	learn: 84.2778855	total: 149ms	remaining: 2.72s
52:	learn: 83.5657173	total: 152ms	remaining: 2.71s
53:	learn: 82.9229521	total: 154ms	remaining: 2.7s
54:	learn: 82.1782116	total: 157ms	remaining: 2.69s
55:	learn: 81.5414071	total: 159ms	remaining: 2.69s
56:	learn: 81.0656023	total: 162ms	remaining: 2.68s
57:	learn: 80.3642380	total: 165ms	remaining: 2.67s
58:	learn: 79.9280196	total: 167ms	remaining: 2.67s
59:	learn: 79.3317870	total: 170ms	remaining: 2.66s
60:	learn: 78.6857278	total: 172ms	remaining: 2.65s
61:	learn: 78.2099306	total: 175ms	remaining: 2.65s
62:	learn: 77.6581997	total: 179ms	remaining: 2.67s
63:	learn: 77.

## 8

In [67]:
catboost_evaluation(train_ft_8, test_ft_8)

  0%|          | 0/4 [00:00<?, ?it/s]

Learning rate set to 0.056075
0:	learn: 170.5336346	test: 169.4376677	best: 169.4376677 (0)	total: 2.75ms	remaining: 2.75s
100:	learn: 68.0847433	test: 87.2084517	best: 86.9889846 (98)	total: 270ms	remaining: 2.41s
200:	learn: 55.1976740	test: 84.0062823	best: 84.0062823 (200)	total: 530ms	remaining: 2.11s
300:	learn: 47.4343677	test: 84.0917170	best: 83.7091664 (253)	total: 803ms	remaining: 1.86s
Stopped by overfitting detector  (120 iterations wait)

bestTest = 83.70916636
bestIteration = 253

Shrink model to first 254 iterations.


 25%|██▌       | 1/4 [00:01<00:03,  1.07s/it]

Learning rate set to 0.056075
0:	learn: 170.0273076	test: 172.2121427	best: 172.2121427 (0)	total: 2.81ms	remaining: 2.81s
100:	learn: 66.5072530	test: 93.5135459	best: 91.9220010 (81)	total: 271ms	remaining: 2.41s


 50%|█████     | 2/4 [00:01<00:01,  1.19it/s]

...

 75%|███████▌  | 3/4 [00:03<00:01,  1.03s/it]

Stopped by overfitting detector  (120 iterations wait)

bestTest = 95.83534932
bestIteration = 72

Shrink model to first 73 iterations.
Learning rate set to 0.056075
0:	learn: 168.3945323	test: 190.0980806	best: 190.0980806 (0)	total: 4.01ms	remaining: 4s
100:	learn: 64.9732586	test: 149.5043083	best: 143.6355854 (41)	total: 529ms	remaining: 4.71s


100%|██████████| 4/4 [00:04<00:00,  1.00s/it]

Stopped by overfitting detector  (120 iterations wait)

bestTest = 143.6355854
bestIteration = 41

Shrink model to first 42 iterations.
Learning rate set to 0.045739
0:	learn: 171.8264696	total: 6.19ms	remaining: 6.18s
1:	learn: 167.0149073	total: 11ms	remaining: 5.49s
2:	learn: 162.4082533	total: 19.2ms	remaining: 6.38s
3:	learn: 157.8824090	total: 26.9ms	remaining: 6.69s
4:	learn: 153.7171445	total: 34.7ms	remaining: 6.91s
5:	learn: 150.0302085	total: 42.7ms	remaining: 7.08s
6:	learn: 146.1067598	total: 51.1ms	remaining: 7.24s
7:	learn: 142.3940784	total: 59.1ms	remaining: 7.33s
8:	learn: 139.2122350	total: 69ms	remaining: 7.6s
9:	learn: 135.9747081	total: 76.1ms	remaining: 7.54s
10:	learn: 133.2266286	total: 80.7ms	remaining: 7.26s
11:	learn: 130.0774001	total: 89.4ms	remaining: 7.36s
12:	learn: 127.3347453	total: 94.1ms	remaining: 7.14s
13:	learn: 124.7471010	total: 102ms	remaining: 7.2s
14:	learn: 122.3161767	total: 108ms	remaining: 7.06s
15:	learn: 119.8798365	total: 113ms	remain

20:	learn: 110.0571782	total: 150ms	remaining: 7.01s
21:	learn: 108.1753308	total: 156ms	remaining: 6.95s
22:	learn: 106.6912116	total: 164ms	remaining: 6.98s
23:	learn: 105.2171029	total: 171ms	remaining: 6.97s
24:	learn: 103.7244994	total: 176ms	remaining: 6.87s
25:	learn: 102.4151697	total: 182ms	remaining: 6.84s
26:	learn: 101.2373367	total: 189ms	remaining: 6.79s
27:	learn: 100.1624749	total: 196ms	remaining: 6.8s
28:	learn: 98.7619760	total: 203ms	remaining: 6.81s
29:	learn: 97.7709287	total: 212ms	remaining: 6.86s
30:	learn: 96.8093815	total: 219ms	remaining: 6.84s
31:	learn: 95.8984030	total: 226ms	remaining: 6.85s
32:	learn: 94.9930747	total: 234ms	remaining: 6.85s
33:	learn: 94.0331818	total: 241ms	remaining: 6.84s
34:	learn: 93.1826340	total: 248ms	remaining: 6.84s
35:	learn: 92.4177189	total: 255ms	remaining: 6.83s
36:	learn: 91.6537150	total: 264ms	remaining: 6.88s
37:	learn: 90.9131961	total: 270ms	remaining: 6.84s
38:	learn: 90.2886228	total: 276ms	remaining: 6.8s
39:	le

## 9

In [68]:
catboost_evaluation(train_ft_9, test_ft_9)

  0%|          | 0/4 [00:00<?, ?it/s]

Learning rate set to 0.056075
0:	learn: 830.0664788	test: 842.4226295	best: 842.4226295 (0)	total: 2.86ms	remaining: 2.86s
100:	learn: 77.2539778	test: 106.7479911	best: 105.3292061 (90)	total: 255ms	remaining: 2.27s


 25%|██▌       | 1/4 [00:00<00:01,  1.67it/s]

200:	learn: 57.9481982	test: 111.1904662	best: 105.3292061 (90)	total: 512ms	remaining: 2.04s
Stopped by overfitting detector  (120 iterations wait)

bestTest = 105.3292061
bestIteration = 90

Shrink model to first 91 iterations.
Learning rate set to 0.056075
0:	learn: 827.6236174	test: 874.9919752	best: 874.9919752 (0)	total: 2.77ms	remaining: 2.77s
100:	learn: 75.4843379	test: 102.0197591	best: 101.8487955 (99)	total: 261ms	remaining: 2.32s
200:	learn: 56.9827861	test: 97.7299950	best: 97.6513360 (199)	total: 519ms	remaining: 2.06s
300:	learn: 46.3131990	test: 97.2972999	best: 97.2619127 (286)	total: 772ms	remaining: 1.79s
400:	learn: 40.1333308	test: 96.9288853	best: 96.8311152 (393)	total: 1.05s	remaining: 1.58s


 50%|█████     | 2/4 [00:02<00:02,  1.13s/it]

...

 75%|███████▌  | 3/4 [00:03<00:01,  1.18s/it]

400:	learn: 41.1529927	test: 108.5694973	best: 107.9128153 (331)	total: 1.03s	remaining: 1.55s
Stopped by overfitting detector  (120 iterations wait)

bestTest = 107.9128153
bestIteration = 331

Shrink model to first 332 iterations.
Learning rate set to 0.056075
0:	learn: 828.4227728	test: 858.2182242	best: 858.2182242 (0)	total: 3.03ms	remaining: 3.03s
100:	learn: 75.7896736	test: 100.4985543	best: 100.4006411 (97)	total: 273ms	remaining: 2.43s


100%|██████████| 4/4 [00:03<00:00,  1.01it/s]

200:	learn: 57.4671700	test: 102.3143561	best: 100.2922328 (101)	total: 525ms	remaining: 2.09s
Stopped by overfitting detector  (120 iterations wait)

bestTest = 100.2922328
bestIteration = 101

Shrink model to first 102 iterations.
Learning rate set to 0.045739
0:	learn: 838.4575389	total: 2.69ms	remaining: 2.69s
1:	learn: 804.5824157	total: 5.46ms	remaining: 2.72s
2:	learn: 772.1689766	total: 8.11ms	remaining: 2.69s
3:	learn: 740.4939190	total: 10.7ms	remaining: 2.67s
4:	learn: 710.2138648	total: 13.3ms	remaining: 2.64s
5:	learn: 681.3755982	total: 15.9ms	remaining: 2.63s
6:	learn: 657.5058583	total: 18.5ms	remaining: 2.62s
7:	learn: 630.6706067	total: 21.2ms	remaining: 2.63s
8:	learn: 606.3866366	total: 23.8ms	remaining: 2.62s
9:	learn: 582.3570214	total: 26.4ms	remaining: 2.61s
10:	learn: 561.3617137	total: 29.2ms	remaining: 2.62s
11:	learn: 540.2018556	total: 31.8ms	remaining: 2.62s
12:	learn: 520.9564819	total: 34.6ms	remaining: 2.62s
13:	learn: 501.3333243	total: 37.2ms	remainin

38:	learn: 206.5500415	total: 107ms	remaining: 2.65s
39:	learn: 200.5191111	total: 110ms	remaining: 2.65s
40:	learn: 194.6501280	total: 113ms	remaining: 2.64s
41:	learn: 189.0136388	total: 116ms	remaining: 2.64s
42:	learn: 183.8256482	total: 118ms	remaining: 2.62s
43:	learn: 178.5292162	total: 121ms	remaining: 2.62s
44:	learn: 173.5178548	total: 123ms	remaining: 2.62s
45:	learn: 169.0929310	total: 126ms	remaining: 2.61s
46:	learn: 164.7016046	total: 129ms	remaining: 2.61s
47:	learn: 160.3482793	total: 131ms	remaining: 2.6s
48:	learn: 156.4439282	total: 134ms	remaining: 2.6s
49:	learn: 152.6205291	total: 137ms	remaining: 2.6s
50:	learn: 149.0124541	total: 140ms	remaining: 2.6s
51:	learn: 145.5023043	total: 142ms	remaining: 2.6s
52:	learn: 142.2664174	total: 145ms	remaining: 2.6s
53:	learn: 139.5250205	total: 148ms	remaining: 2.59s
54:	learn: 136.6899609	total: 151ms	remaining: 2.59s
55:	learn: 134.1117558	total: 153ms	remaining: 2.58s
56:	learn: 131.5229764	total: 156ms	remaining: 2.58s

## 10

In [69]:
catboost_evaluation(train_ft_10, test_ft_10)

  0%|          | 0/4 [00:00<?, ?it/s]

Learning rate set to 0.056075
0:	learn: 567.6203044	test: 463.8723316	best: 463.8723316 (0)	total: 3ms	remaining: 3s


 25%|██▌       | 1/4 [00:00<00:01,  2.22it/s]

100:	learn: 180.8799768	test: 449.6441499	best: 417.3293287 (20)	total: 272ms	remaining: 2.42s
Stopped by overfitting detector  (120 iterations wait)

bestTest = 417.3293287
bestIteration = 20

Shrink model to first 21 iterations.
Learning rate set to 0.056075
0:	learn: 571.0095561	test: 425.8898800	best: 425.8898800 (0)	total: 2.99ms	remaining: 2.99s
100:	learn: 172.9597888	test: 354.4111260	best: 334.4822505 (25)	total: 457ms	remaining: 4.07s


 50%|█████     | 2/4 [00:01<00:01,  1.65it/s]

...

 75%|███████▌  | 3/4 [00:02<00:00,  1.18it/s]

Stopped by overfitting detector  (120 iterations wait)

bestTest = 252.2905509
bestIteration = 63

Shrink model to first 64 iterations.
Learning rate set to 0.056075
0:	learn: 571.2408849	test: 411.5376694	best: 411.5376694 (0)	total: 6.35ms	remaining: 6.35s
100:	learn: 183.3272778	test: 382.3376698	best: 331.9283539 (27)	total: 570ms	remaining: 5.07s


100%|██████████| 4/4 [00:03<00:00,  1.24it/s]

Stopped by overfitting detector  (120 iterations wait)

bestTest = 331.9283539
bestIteration = 27

Shrink model to first 28 iterations.
Learning rate set to 0.045739
0:	learn: 563.7147246	total: 6.52ms	remaining: 6.51s
1:	learn: 547.7699936	total: 15.5ms	remaining: 7.72s
2:	learn: 532.9894888	total: 21.9ms	remaining: 7.26s
3:	learn: 519.1523849	total: 29.4ms	remaining: 7.32s
4:	learn: 506.7100568	total: 38.2ms	remaining: 7.59s
5:	learn: 494.5030535	total: 43.3ms	remaining: 7.17s
6:	learn: 483.0616083	total: 51.2ms	remaining: 7.26s
7:	learn: 471.8601098	total: 58.6ms	remaining: 7.27s
8:	learn: 459.3345175	total: 66.3ms	remaining: 7.3s
9:	learn: 448.5728543	total: 70.8ms	remaining: 7.01s
10:	learn: 436.9515485	total: 75.2ms	remaining: 6.76s
11:	learn: 428.9598626	total: 81.3ms	remaining: 6.69s
12:	learn: 420.6518894	total: 86.1ms	remaining: 6.53s
13:	learn: 411.6142269	total: 93.9ms	remaining: 6.62s
14:	learn: 402.3558640	total: 101ms	remaining: 6.66s
15:	learn: 394.7724805	total: 109ms	

22:	learn: 350.1921566	total: 163ms	remaining: 6.92s
23:	learn: 345.6917039	total: 171ms	remaining: 6.95s
24:	learn: 340.4898740	total: 184ms	remaining: 7.18s
25:	learn: 334.7977233	total: 190ms	remaining: 7.12s
26:	learn: 329.5375270	total: 198ms	remaining: 7.15s
27:	learn: 324.6993330	total: 206ms	remaining: 7.17s
28:	learn: 320.7334190	total: 215ms	remaining: 7.19s
29:	learn: 315.4490407	total: 223ms	remaining: 7.2s
30:	learn: 310.8416079	total: 231ms	remaining: 7.22s
31:	learn: 307.3351674	total: 239ms	remaining: 7.24s
32:	learn: 303.7582096	total: 244ms	remaining: 7.16s
33:	learn: 298.9795779	total: 248ms	remaining: 7.05s
34:	learn: 294.4956321	total: 253ms	remaining: 6.97s
35:	learn: 291.5565026	total: 257ms	remaining: 6.89s
36:	learn: 287.4863395	total: 262ms	remaining: 6.82s
37:	learn: 284.7076296	total: 265ms	remaining: 6.7s
38:	learn: 281.4176250	total: 271ms	remaining: 6.69s
39:	learn: 278.5054150	total: 276ms	remaining: 6.63s
40:	learn: 275.1707446	total: 289ms	remaining: 6

## 11

In [70]:
catboost_evaluation(train_ft_11, test_ft_11)

  0%|          | 0/4 [00:00<?, ?it/s]

Learning rate set to 0.056075
0:	learn: 299.7999550	test: 283.2193689	best: 283.2193689 (0)	total: 2.87ms	remaining: 2.87s
100:	learn: 69.8085634	test: 90.0704778	best: 90.0704778 (100)	total: 272ms	remaining: 2.42s
200:	learn: 55.7744201	test: 88.6000268	best: 88.4552397 (174)	total: 537ms	remaining: 2.13s
300:	learn: 46.6921445	test: 88.1272933	best: 87.6794298 (272)	total: 781ms	remaining: 1.81s
400:	learn: 40.4740368	test: 87.2034633	best: 87.1616690 (391)	total: 1.03s	remaining: 1.54s
500:	learn: 35.6334300	test: 86.8716852	best: 86.7395761 (468)	total: 1.29s	remaining: 1.28s
600:	learn: 31.8660768	test: 86.9577861	best: 86.6228630 (547)	total: 1.56s	remaining: 1.04s
Stopped by overfitting detector  (120 iterations wait)

bestTest = 86.62286302
bestIteration = 547

Shrink model to first 548 iterations.


 25%|██▌       | 1/4 [00:01<00:05,  1.84s/it]

Learning rate set to 0.056075
0:	learn: 299.2852562	test: 290.0663091	best: 290.0663091 (0)	total: 2.92ms	remaining: 2.92s
100:	learn: 69.4741382	test: 95.0521484	best: 94.4841548 (82)	total: 251ms	remaining: 2.24s


 50%|█████     | 2/4 [00:02<00:02,  1.09s/it]

...

 75%|███████▌  | 3/4 [00:02<00:00,  1.17it/s]

Stopped by overfitting detector  (120 iterations wait)

bestTest = 151.2623792
bestIteration = 72

Shrink model to first 73 iterations.
Learning rate set to 0.056075
0:	learn: 298.0776029	test: 304.6017469	best: 304.6017469 (0)	total: 4.13ms	remaining: 4.12s
100:	learn: 68.8973570	test: 100.8934419	best: 99.7586965 (69)	total: 276ms	remaining: 2.46s


100%|██████████| 4/4 [00:03<00:00,  1.13it/s]

Stopped by overfitting detector  (120 iterations wait)

bestTest = 99.75869649
bestIteration = 69

Shrink model to first 70 iterations.
Learning rate set to 0.045739
0:	learn: 301.2763434	total: 2.85ms	remaining: 2.85s
1:	learn: 290.4681911	total: 5.4ms	remaining: 2.69s
2:	learn: 280.0815307	total: 7.87ms	remaining: 2.62s
3:	learn: 269.8120660	total: 10.4ms	remaining: 2.59s
4:	learn: 260.3074210	total: 12.8ms	remaining: 2.55s
5:	learn: 250.9776973	total: 15.3ms	remaining: 2.53s
6:	learn: 242.5906554	total: 17.7ms	remaining: 2.51s
7:	learn: 234.8214226	total: 20.5ms	remaining: 2.54s
8:	learn: 227.2701756	total: 23ms	remaining: 2.53s
9:	learn: 219.5140026	total: 25.8ms	remaining: 2.55s
10:	learn: 212.4101802	total: 28.3ms	remaining: 2.55s
11:	learn: 206.0020522	total: 31ms	remaining: 2.55s
12:	learn: 199.9300337	total: 33.6ms	remaining: 2.55s
13:	learn: 193.8601530	total: 36.2ms	remaining: 2.55s
14:	learn: 187.8143290	total: 39.1ms	remaining: 2.57s
15:	learn: 182.0800076	total: 41.9ms	re

57:	learn: 90.6136266	total: 159ms	remaining: 2.58s
58:	learn: 90.0056518	total: 163ms	remaining: 2.6s
59:	learn: 89.3775954	total: 166ms	remaining: 2.59s
60:	learn: 88.8870742	total: 168ms	remaining: 2.59s
61:	learn: 88.3347763	total: 171ms	remaining: 2.58s
62:	learn: 87.8932301	total: 173ms	remaining: 2.58s
63:	learn: 87.3706921	total: 176ms	remaining: 2.58s
64:	learn: 86.8189122	total: 182ms	remaining: 2.62s
65:	learn: 86.1938640	total: 185ms	remaining: 2.61s
66:	learn: 85.6507414	total: 187ms	remaining: 2.61s
67:	learn: 85.1171441	total: 190ms	remaining: 2.6s
68:	learn: 84.6622719	total: 193ms	remaining: 2.6s
69:	learn: 84.3768101	total: 196ms	remaining: 2.6s
70:	learn: 84.0087520	total: 198ms	remaining: 2.59s
71:	learn: 83.5943277	total: 201ms	remaining: 2.59s
72:	learn: 83.1972815	total: 204ms	remaining: 2.59s
73:	learn: 82.7464934	total: 207ms	remaining: 2.58s
74:	learn: 82.3544815	total: 209ms	remaining: 2.58s
75:	learn: 82.0464855	total: 212ms	remaining: 2.58s
76:	learn: 81.71

## 12

In [71]:
catboost_evaluation(train_ft_12, test_ft_12)

  0%|          | 0/4 [00:00<?, ?it/s]

Learning rate set to 0.056075
0:	learn: 264.2540150	test: 272.0780122	best: 272.0780122 (0)	total: 3.11ms	remaining: 3.11s
100:	learn: 62.0774345	test: 87.3086525	best: 86.6553226 (66)	total: 253ms	remaining: 2.25s


 25%|██▌       | 1/4 [00:00<00:01,  1.89it/s]

Stopped by overfitting detector  (120 iterations wait)

bestTest = 86.6553226
bestIteration = 66

Shrink model to first 67 iterations.
Learning rate set to 0.056075
0:	learn: 265.7836643	test: 254.5724529	best: 254.5724529 (0)	total: 3.05ms	remaining: 3.04s
100:	learn: 61.7290039	test: 73.3734496	best: 68.0817443 (66)	total: 272ms	remaining: 2.42s


 50%|█████     | 2/4 [00:01<00:01,  1.84it/s]

...

 75%|███████▌  | 3/4 [00:02<00:00,  1.27it/s]

Stopped by overfitting detector  (120 iterations wait)

bestTest = 121.1310542
bestIteration = 65

Shrink model to first 66 iterations.
Learning rate set to 0.056075
0:	learn: 268.7944415	test: 213.2935200	best: 213.2935200 (0)	total: 6.79ms	remaining: 6.79s
100:	learn: 57.9688914	test: 161.3157282	best: 126.8938944 (21)	total: 599ms	remaining: 5.33s


100%|██████████| 4/4 [00:03<00:00,  1.31it/s]

Stopped by overfitting detector  (120 iterations wait)

bestTest = 126.8938944
bestIteration = 21

Shrink model to first 22 iterations.
Learning rate set to 0.045739
0:	learn: 267.0212503	total: 3.84ms	remaining: 3.84s
1:	learn: 257.2932794	total: 7.22ms	remaining: 3.6s
2:	learn: 248.4234700	total: 10ms	remaining: 3.33s
3:	learn: 239.7283640	total: 13.2ms	remaining: 3.29s
4:	learn: 231.6992940	total: 16.8ms	remaining: 3.35s
5:	learn: 223.7533896	total: 20.3ms	remaining: 3.37s
6:	learn: 217.3285675	total: 23.8ms	remaining: 3.38s
7:	learn: 210.2990467	total: 26.9ms	remaining: 3.33s
8:	learn: 203.5531842	total: 29.8ms	remaining: 3.28s
9:	learn: 196.8448160	total: 33.3ms	remaining: 3.29s
10:	learn: 190.8800003	total: 36.8ms	remaining: 3.31s
11:	learn: 185.1752745	total: 39.8ms	remaining: 3.28s
12:	learn: 180.0219538	total: 43.5ms	remaining: 3.3s
13:	learn: 174.6472820	total: 47.3ms	remaining: 3.33s
14:	learn: 169.8346362	total: 51ms	remaining: 3.35s
15:	learn: 165.4575471	total: 53.8ms	rem

33:	learn: 113.4014785	total: 155ms	remaining: 4.41s
34:	learn: 111.4026656	total: 161ms	remaining: 4.45s
35:	learn: 109.5733203	total: 174ms	remaining: 4.66s
36:	learn: 108.1140097	total: 179ms	remaining: 4.65s
37:	learn: 106.6712905	total: 187ms	remaining: 4.72s
38:	learn: 105.3412430	total: 193ms	remaining: 4.75s
39:	learn: 104.0183051	total: 200ms	remaining: 4.81s
40:	learn: 102.7518636	total: 208ms	remaining: 4.86s
41:	learn: 101.6767447	total: 216ms	remaining: 4.94s
42:	learn: 100.0869526	total: 224ms	remaining: 4.99s
43:	learn: 99.1159302	total: 234ms	remaining: 5.08s
44:	learn: 97.9409599	total: 240ms	remaining: 5.09s
45:	learn: 96.7806954	total: 248ms	remaining: 5.14s
46:	learn: 95.9670286	total: 256ms	remaining: 5.18s
47:	learn: 94.8341007	total: 263ms	remaining: 5.22s
48:	learn: 93.7911805	total: 271ms	remaining: 5.26s
49:	learn: 92.5912099	total: 279ms	remaining: 5.3s
50:	learn: 91.8872420	total: 287ms	remaining: 5.34s
51:	learn: 90.8932176	total: 294ms	remaining: 5.36s
52:

## 13

In [ ]:
catboost_evaluation(train_ft_13, test_ft_13)

  0%|          | 0/4 [00:00<?, ?it/s]

Learning rate set to 0.056075
0:	learn: 337.1297067	test: 250.3830217	best: 250.3830217 (0)	total: 3.15ms	remaining: 3.14s


 25%|██▌       | 1/4 [00:00<00:01,  2.54it/s]

100:	learn: 148.8690115	test: 167.0917755	best: 157.2375105 (49)	total: 243ms	remaining: 2.17s
Stopped by overfitting detector  (120 iterations wait)

bestTest = 157.2375105
bestIteration = 49

Shrink model to first 50 iterations.
Learning rate set to 0.056075
0:	learn: 332.3277129	test: 307.4688747	best: 307.4688747 (0)	total: 2.64ms	remaining: 2.64s
100:	learn: 143.7172902	test: 206.8716688	best: 206.6479218 (87)	total: 281ms	remaining: 2.5s
200:	learn: 113.2708434	test: 207.0724133	best: 205.7561393 (152)	total: 555ms	remaining: 2.21s
300:	learn: 94.0638430	test: 206.4032017	best: 205.2142905 (268)	total: 846ms	remaining: 1.96s


 50%|█████     | 2/4 [00:01<00:01,  1.17it/s]

...

 75%|███████▌  | 3/4 [00:01<00:00,  1.62it/s]

100:	learn: 144.8053363	test: 222.7588997	best: 189.0981300 (26)	total: 193ms	remaining: 1.72s
Stopped by overfitting detector  (120 iterations wait)

bestTest = 189.09813
bestIteration = 26

Shrink model to first 27 iterations.
Learning rate set to 0.056075
0:	learn: 328.1537532	test: 363.9895791	best: 363.9895791 (0)	total: 1.15ms	remaining: 1.15s


100%|██████████| 4/4 [00:02<00:00,  1.89it/s]

100:	learn: 145.9206878	test: 298.4854685	best: 265.7361281 (37)	total: 107ms	remaining: 949ms
Stopped by overfitting detector  (120 iterations wait)

bestTest = 265.7361281
bestIteration = 37

Shrink model to first 38 iterations.
Learning rate set to 0.045739
0:	learn: 332.3510347	total: 1.56ms	remaining: 1.56s
1:	learn: 324.1423846	total: 3.35ms	remaining: 1.67s
2:	learn: 316.3900839	total: 6.17ms	remaining: 2.05s
3:	learn: 309.2668962	total: 11.4ms	remaining: 2.83s
4:	learn: 302.7548047	total: 14.4ms	remaining: 2.87s
5:	learn: 296.5722616	total: 17.9ms	remaining: 2.97s
6:	learn: 290.8170065	total: 20ms	remaining: 2.83s
7:	learn: 285.1058543	total: 21ms	remaining: 2.61s
8:	learn: 279.9308177	total: 22.1ms	remaining: 2.43s
9:	learn: 274.5297470	total: 23.2ms	remaining: 2.3s
10:	learn: 269.7323495	total: 24.4ms	remaining: 2.19s
11:	learn: 265.6810371	total: 25.5ms	remaining: 2.1s
12:	learn: 262.0561654	total: 26.7ms	remaining: 2.03s
13:	learn: 257.9716228	total: 28.2ms	remaining: 1.98s

71:	learn: 170.0449268	total: 97.3ms	remaining: 1.25s
72:	learn: 169.4100123	total: 98.2ms	remaining: 1.25s
73:	learn: 168.8226102	total: 99.1ms	remaining: 1.24s
74:	learn: 168.3116610	total: 100ms	remaining: 1.23s
75:	learn: 167.7891084	total: 101ms	remaining: 1.23s
76:	learn: 167.4179862	total: 102ms	remaining: 1.22s
77:	learn: 167.1283700	total: 103ms	remaining: 1.21s
78:	learn: 166.1345171	total: 104ms	remaining: 1.21s
79:	learn: 165.8477290	total: 105ms	remaining: 1.21s
80:	learn: 165.3432079	total: 106ms	remaining: 1.2s
81:	learn: 164.7847295	total: 107ms	remaining: 1.2s
82:	learn: 164.3519391	total: 108ms	remaining: 1.19s
83:	learn: 163.7315543	total: 109ms	remaining: 1.19s
84:	learn: 163.2740901	total: 110ms	remaining: 1.18s
85:	learn: 163.0710464	total: 111ms	remaining: 1.18s
86:	learn: 162.6659624	total: 112ms	remaining: 1.17s
87:	learn: 162.4603805	total: 113ms	remaining: 1.17s
88:	learn: 162.1762521	total: 114ms	remaining: 1.16s
89:	learn: 161.6833712	total: 115ms	remaining

## 14

In [ ]:
catboost_evaluation(train_ft_14, test_ft_14)

  0%|          | 0/4 [00:00<?, ?it/s]

Learning rate set to 0.056075
0:	learn: 432.6433022	test: 427.4797999	best: 427.4797999 (0)	total: 1.01ms	remaining: 1.01s


 25%|██▌       | 1/4 [00:00<00:00,  5.26it/s]

100:	learn: 260.5956723	test: 396.7204528	best: 350.8113887 (27)	total: 101ms	remaining: 900ms
Stopped by overfitting detector  (120 iterations wait)

bestTest = 350.8113887
bestIteration = 27

Shrink model to first 28 iterations.
Learning rate set to 0.056075
0:	learn: 427.2035239	test: 480.8930787	best: 480.8930787 (0)	total: 986us	remaining: 985ms
100:	learn: 258.5555703	test: 381.8118625	best: 376.1911008 (43)	total: 96.9ms	remaining: 863ms


 50%|█████     | 2/4 [00:00<00:00,  5.04it/s]

...

 75%|███████▌  | 3/4 [00:00<00:00,  5.30it/s]

Stopped by overfitting detector  (120 iterations wait)

bestTest = 340.7480319
bestIteration = 21

Shrink model to first 22 iterations.
Learning rate set to 0.056075
0:	learn: 429.5938334	test: 463.8153151	best: 463.8153151 (0)	total: 942us	remaining: 942ms
100:	learn: 265.2519263	test: 381.4565279	best: 347.6645864 (39)	total: 103ms	remaining: 917ms


100%|██████████| 4/4 [00:00<00:00,  5.10it/s]

Stopped by overfitting detector  (120 iterations wait)

bestTest = 347.6645864
bestIteration = 39

Shrink model to first 40 iterations.
Learning rate set to 0.045739
0:	learn: 433.9677759	total: 1.18ms	remaining: 1.18s
1:	learn: 426.7259600	total: 2.23ms	remaining: 1.11s
2:	learn: 419.6361460	total: 3.33ms	remaining: 1.11s
3:	learn: 412.4757850	total: 4.37ms	remaining: 1.09s
4:	learn: 406.3866451	total: 5.39ms	remaining: 1.07s
5:	learn: 400.4982527	total: 6.51ms	remaining: 1.08s
6:	learn: 395.1414215	total: 7.55ms	remaining: 1.07s
7:	learn: 390.5166002	total: 8.69ms	remaining: 1.08s
8:	learn: 385.5018665	total: 9.67ms	remaining: 1.06s
9:	learn: 380.5567255	total: 10.7ms	remaining: 1.06s
10:	learn: 376.3454278	total: 11.9ms	remaining: 1.07s
11:	learn: 372.4347813	total: 13.1ms	remaining: 1.07s
12:	learn: 368.7710745	total: 14.3ms	remaining: 1.08s
13:	learn: 364.7945358	total: 15.5ms	remaining: 1.09s
14:	learn: 361.3141830	total: 16.7ms	remaining: 1.1s
15:	learn: 358.3300205	total: 18.1m

118:	learn: 269.1419737	total: 146ms	remaining: 1.08s
119:	learn: 268.4765663	total: 148ms	remaining: 1.09s
120:	learn: 268.3463147	total: 150ms	remaining: 1.09s
121:	learn: 267.8474986	total: 152ms	remaining: 1.09s
122:	learn: 267.6272901	total: 153ms	remaining: 1.09s
123:	learn: 267.1438226	total: 154ms	remaining: 1.09s
124:	learn: 266.6573027	total: 155ms	remaining: 1.08s
125:	learn: 266.2016626	total: 157ms	remaining: 1.08s
126:	learn: 265.7846026	total: 158ms	remaining: 1.08s
127:	learn: 265.4340519	total: 159ms	remaining: 1.08s
128:	learn: 265.0725982	total: 160ms	remaining: 1.08s
129:	learn: 264.3857588	total: 161ms	remaining: 1.08s
130:	learn: 264.0053336	total: 162ms	remaining: 1.08s
131:	learn: 263.8306313	total: 165ms	remaining: 1.08s
132:	learn: 263.2613304	total: 166ms	remaining: 1.08s
133:	learn: 263.0331208	total: 167ms	remaining: 1.08s
134:	learn: 262.7899230	total: 169ms	remaining: 1.08s
135:	learn: 262.6024942	total: 170ms	remaining: 1.08s
136:	learn: 262.3719868	tota

## 15

In [ ]:
catboost_evaluation(train_ft_15, test_ft_15)

  0%|          | 0/4 [00:00<?, ?it/s]

Learning rate set to 0.056075
0:	learn: 233.1510608	test: 212.0893275	best: 212.0893275 (0)	total: 1.16ms	remaining: 1.16s
100:	learn: 58.0113003	test: 83.2262037	best: 82.8350238 (95)	total: 102ms	remaining: 911ms


 25%|██▌       | 1/4 [00:00<00:00,  3.62it/s]

200:	learn: 44.3890294	test: 93.7032562	best: 82.8350238 (95)	total: 207ms	remaining: 823ms
Stopped by overfitting detector  (120 iterations wait)

bestTest = 82.83502375
bestIteration = 95

Shrink model to first 96 iterations.
Learning rate set to 0.056075
0:	learn: 232.8373147	test: 215.2853350	best: 215.2853350 (0)	total: 1.05ms	remaining: 1.05s
100:	learn: 55.6429708	test: 129.3981554	best: 123.8990994 (50)	total: 98.3ms	remaining: 875ms


 50%|█████     | 2/4 [00:00<00:00,  3.98it/s]

...

 75%|███████▌  | 3/4 [00:00<00:00,  3.98it/s]

Stopped by overfitting detector  (120 iterations wait)

bestTest = 97.32275657
bestIteration = 67

Shrink model to first 68 iterations.
Learning rate set to 0.056075
0:	learn: 230.4135918	test: 243.9848228	best: 243.9848228 (0)	total: 1.04ms	remaining: 1.04s
100:	learn: 56.1380668	test: 98.0078008	best: 98.0078008 (100)	total: 99.8ms	remaining: 888ms


100%|██████████| 4/4 [00:01<00:00,  3.84it/s]

200:	learn: 41.7859556	test: 98.6944175	best: 97.9735000 (104)	total: 197ms	remaining: 783ms
Stopped by overfitting detector  (120 iterations wait)

bestTest = 97.97350002
bestIteration = 104

Shrink model to first 105 iterations.
Learning rate set to 0.045739
0:	learn: 232.8591003	total: 3.84ms	remaining: 3.83s
1:	learn: 224.8752921	total: 5.21ms	remaining: 2.6s
2:	learn: 217.3101931	total: 6.42ms	remaining: 2.13s
3:	learn: 209.9185186	total: 7.55ms	remaining: 1.88s
4:	learn: 203.2804070	total: 8.79ms	remaining: 1.75s
5:	learn: 196.7350736	total: 9.97ms	remaining: 1.65s
6:	learn: 190.3704355	total: 11.1ms	remaining: 1.58s
7:	learn: 184.6693735	total: 12.3ms	remaining: 1.53s
8:	learn: 178.7858100	total: 13.4ms	remaining: 1.48s
9:	learn: 173.5331529	total: 14.6ms	remaining: 1.45s
10:	learn: 168.3587299	total: 15.8ms	remaining: 1.42s
11:	learn: 163.2624210	total: 17ms	remaining: 1.4s
12:	learn: 158.6104187	total: 18.2ms	remaining: 1.39s
13:	learn: 154.2180293	total: 19.4ms	remaining: 1.3

99:	learn: 62.6380302	total: 125ms	remaining: 1.13s
100:	learn: 62.4188662	total: 127ms	remaining: 1.13s
101:	learn: 62.2902477	total: 129ms	remaining: 1.14s
102:	learn: 62.1625229	total: 130ms	remaining: 1.13s
103:	learn: 61.9719383	total: 131ms	remaining: 1.13s
104:	learn: 61.7491385	total: 133ms	remaining: 1.13s
105:	learn: 61.5693182	total: 134ms	remaining: 1.13s
106:	learn: 61.3887553	total: 135ms	remaining: 1.13s
107:	learn: 61.1930391	total: 136ms	remaining: 1.13s
108:	learn: 61.0328976	total: 138ms	remaining: 1.12s
109:	learn: 60.9374005	total: 139ms	remaining: 1.12s
110:	learn: 60.7446557	total: 140ms	remaining: 1.12s
111:	learn: 60.4893073	total: 141ms	remaining: 1.12s
112:	learn: 60.3324927	total: 142ms	remaining: 1.12s
113:	learn: 60.1750624	total: 144ms	remaining: 1.12s
114:	learn: 60.0199016	total: 145ms	remaining: 1.11s
115:	learn: 59.8615743	total: 146ms	remaining: 1.11s
116:	learn: 59.6940488	total: 147ms	remaining: 1.11s
117:	learn: 59.3976220	total: 149ms	remaining: 

## 16

In [ ]:
catboost_evaluation(train_ft_16, test_ft_16)

## 17

In [ ]:
catboost_evaluation(train_ft_17, test_ft_17)

  0%|          | 0/4 [00:00<?, ?it/s]

Learning rate set to 0.056075
0:	learn: 419.1237732	test: 397.0054798	best: 397.0054798 (0)	total: 2.74ms	remaining: 2.74s
100:	learn: 58.4711008	test: 87.8348150	best: 87.8348150 (100)	total: 255ms	remaining: 2.27s


 25%|██▌       | 1/4 [00:00<00:01,  1.56it/s]

200:	learn: 42.3884344	test: 88.8786465	best: 87.5138494 (103)	total: 520ms	remaining: 2.07s
Stopped by overfitting detector  (120 iterations wait)

bestTest = 87.51384945
bestIteration = 103

Shrink model to first 104 iterations.
Learning rate set to 0.056075
0:	learn: 418.9778414	test: 399.1819732	best: 399.1819732 (0)	total: 2.91ms	remaining: 2.9s
100:	learn: 58.6806820	test: 177.3476842	best: 169.4923363 (49)	total: 261ms	remaining: 2.32s
Stopped by overfitting detector  (120 iterations wait)

bestTest = 169.4923363
bestIteration = 49

Shrink model to first 50 iterations.


 50%|█████     | 2/4 [00:01<00:01,  1.82it/s]

...

 75%|███████▌  | 3/4 [00:01<00:00,  1.76it/s]

200:	learn: 44.8494619	test: 73.0818392	best: 71.5004449 (87)	total: 521ms	remaining: 2.07s
Stopped by overfitting detector  (120 iterations wait)

bestTest = 71.50044489
bestIteration = 87

Shrink model to first 88 iterations.
Learning rate set to 0.056075
0:	learn: 416.6847133	test: 436.0740539	best: 436.0740539 (0)	total: 2.68ms	remaining: 2.68s
100:	learn: 58.5733576	test: 77.3356343	best: 77.3356343 (100)	total: 279ms	remaining: 2.48s


100%|██████████| 4/4 [00:02<00:00,  1.61it/s]

200:	learn: 44.1750333	test: 77.1173294	best: 74.4177173 (142)	total: 554ms	remaining: 2.2s
Stopped by overfitting detector  (120 iterations wait)

bestTest = 74.41771734
bestIteration = 142

Shrink model to first 143 iterations.
Learning rate set to 0.045739


0:	learn: 421.1968620	total: 7.87ms	remaining: 7.87s
1:	learn: 404.0367820	total: 11.5ms	remaining: 5.73s
2:	learn: 387.6720895	total: 14.3ms	remaining: 4.77s
3:	learn: 372.6494864	total: 17.1ms	remaining: 4.27s
4:	learn: 357.8367654	total: 20.2ms	remaining: 4.03s
5:	learn: 344.0426611	total: 22.4ms	remaining: 3.7s
6:	learn: 330.4700881	total: 25.3ms	remaining: 3.59s
7:	learn: 317.7293417	total: 28.3ms	remaining: 3.51s
8:	learn: 305.4434943	total: 31.1ms	remaining: 3.42s
9:	learn: 293.8043918	total: 34ms	remaining: 3.36s
10:	learn: 283.0720971	total: 36.8ms	remaining: 3.31s
11:	learn: 272.3311260	total: 39.6ms	remaining: 3.26s
12:	learn: 262.2796894	total: 42.3ms	remaining: 3.21s
13:	learn: 253.0493794	total: 44ms	remaining: 3.1s
14:	learn: 243.4378048	total: 46.8ms	remaining: 3.07s
15:	learn: 234.9511749	total: 49.6ms	remaining: 3.05s
16:	learn: 226.5759053	total: 52.5ms	remaining: 3.04s
17:	learn: 218.9269837	total: 55.2ms	remaining: 3.01s
18:	learn: 211.1942106	total: 57.9ms	remaini

## 18

In [ ]:
catboost_evaluation(train_ft_18, test_ft_18)

## 19

In [ ]:
catboost_evaluation(train_ft_19, test_ft_19)

## 20

In [ ]:
catboost_evaluation(train_ft_20, test_ft_20)

## 21

In [ ]:
catboost_evaluation(train_ft_21, test_ft_21)

## 22

In [ ]:
catboost_evaluation(train_ft_22, test_ft_22)

## 23

In [ ]:
catboost_evaluation(train_ft_23, test_ft_23)

## 24

In [ ]:
catboost_evaluation(train_ft_24, test_ft_24)

## 25

In [ ]:
catboost_evaluation(train_ft_25, test_ft_25)

  0%|          | 0/4 [00:00<?, ?it/s]

Learning rate set to 0.056075
0:	learn: 2259.8798414	test: 2325.9790382	best: 2325.9790382 (0)	total: 2.58ms	remaining: 2.58s
100:	learn: 289.8054267	test: 592.4012262	best: 512.4310445 (48)	total: 268ms	remaining: 2.38s
Stopped by overfitting detector  (120 iterations wait)

bestTest = 512.4310445
bestIteration = 48

Shrink model to first 49 iterations.


 25%|██▌       | 1/4 [00:00<00:01,  1.94it/s]

Learning rate set to 0.056075
0:	learn: 2265.1825003	test: 2180.5020298	best: 2180.5020298 (0)	total: 2.89ms	remaining: 2.88s


 50%|█████     | 2/4 [00:00<00:00,  2.08it/s]

...

 75%|███████▌  | 3/4 [00:01<00:00,  2.28it/s]

100:	learn: 273.7146896	test: 883.3497904	best: 788.5090684 (54)	total: 232ms	remaining: 2.07s
Stopped by overfitting detector  (120 iterations wait)

bestTest = 788.5090684
bestIteration = 54

Shrink model to first 55 iterations.
Learning rate set to 0.056075
0:	learn: 2248.8096806	test: 2452.7775332	best: 2452.7775332 (0)	total: 1.05ms	remaining: 1.05s
100:	learn: 294.0317612	test: 394.1936508	best: 394.1936508 (100)	total: 92.3ms	remaining: 822ms
200:	learn: 202.9329038	test: 338.8348762	best: 338.5834894 (197)	total: 188ms	remaining: 747ms
300:	learn: 163.9766817	test: 320.4132850	best: 320.0562903 (293)	total: 286ms	remaining: 663ms
400:	learn: 138.6172259	test: 312.9528805	best: 312.9528805 (400)	total: 382ms	remaining: 570ms
500:	learn: 119.8756209	test: 308.5487332	best: 308.5487332 (500)	total: 469ms	remaining: 467ms


100%|██████████| 4/4 [00:02<00:00,  1.92it/s]

600:	learn: 105.2281883	test: 309.5786880	best: 308.2125155 (535)	total: 575ms	remaining: 382ms
Stopped by overfitting detector  (120 iterations wait)

bestTest = 308.2125155
bestIteration = 535

Shrink model to first 536 iterations.
Learning rate set to 0.045739
0:	learn: 2285.6125388	total: 1.29ms	remaining: 1.29s
1:	learn: 2197.0184189	total: 2.52ms	remaining: 1.26s
2:	learn: 2114.9682339	total: 3.73ms	remaining: 1.24s
3:	learn: 2032.8701886	total: 5.08ms	remaining: 1.26s
4:	learn: 1953.8709821	total: 6.52ms	remaining: 1.3s
5:	learn: 1879.8650114	total: 7.73ms	remaining: 1.28s
6:	learn: 1814.6356575	total: 9.02ms	remaining: 1.28s
7:	learn: 1743.7525390	total: 10.2ms	remaining: 1.26s
8:	learn: 1685.1578110	total: 11.3ms	remaining: 1.25s
9:	learn: 1622.5379654	total: 12.5ms	remaining: 1.24s
10:	learn: 1562.0484266	total: 13.7ms	remaining: 1.23s
11:	learn: 1502.6583548	total: 14.8ms	remaining: 1.22s
12:	learn: 1446.6368593	total: 16ms	remaining: 1.21s
13:	learn: 1398.2477052	total: 17.

71:	learn: 390.1494496	total: 94ms	remaining: 1.21s
72:	learn: 387.2859830	total: 95.2ms	remaining: 1.21s
73:	learn: 382.9090538	total: 96.4ms	remaining: 1.21s
74:	learn: 377.7090346	total: 97.7ms	remaining: 1.2s
75:	learn: 375.2970158	total: 98.9ms	remaining: 1.2s
76:	learn: 372.9143483	total: 100ms	remaining: 1.2s
77:	learn: 369.2419575	total: 101ms	remaining: 1.2s
78:	learn: 366.6312953	total: 102ms	remaining: 1.19s
79:	learn: 364.1227446	total: 103ms	remaining: 1.19s
80:	learn: 361.8850969	total: 105ms	remaining: 1.19s
81:	learn: 358.7325746	total: 106ms	remaining: 1.19s
82:	learn: 356.1499651	total: 107ms	remaining: 1.18s
83:	learn: 353.1197597	total: 108ms	remaining: 1.18s
84:	learn: 351.4981390	total: 110ms	remaining: 1.18s
85:	learn: 349.7295034	total: 111ms	remaining: 1.18s
86:	learn: 347.0464679	total: 112ms	remaining: 1.17s
87:	learn: 344.7586829	total: 113ms	remaining: 1.17s
88:	learn: 343.1810615	total: 114ms	remaining: 1.17s
89:	learn: 340.8945526	total: 116ms	remaining: 

## 26

In [ ]:
catboost_evaluation(train_ft_26, test_ft_26)

  0%|          | 0/4 [00:00<?, ?it/s]

Learning rate set to 0.056075
0:	learn: 939.4489866	test: 884.0336706	best: 884.0336706 (0)	total: 1ms	remaining: 1s
100:	learn: 117.4582368	test: 122.6367669	best: 121.4789935 (91)	total: 105ms	remaining: 931ms
200:	learn: 86.3105198	test: 110.2895604	best: 109.5328849 (191)	total: 202ms	remaining: 802ms
300:	learn: 70.3072544	test: 107.1383441	best: 107.1383441 (300)	total: 297ms	remaining: 690ms
400:	learn: 59.9785017	test: 106.2514440	best: 106.0325210 (362)	total: 404ms	remaining: 604ms


 25%|██▌       | 1/4 [00:00<00:01,  1.77it/s]

Stopped by overfitting detector  (120 iterations wait)

bestTest = 106.032521
bestIteration = 362

Shrink model to first 363 iterations.
Learning rate set to 0.056075
0:	learn: 936.8043192	test: 893.4392336	best: 893.4392336 (0)	total: 1.05ms	remaining: 1.05s
100:	learn: 114.7624154	test: 343.0369089	best: 327.3668674 (47)	total: 110ms	remaining: 976ms


 50%|█████     | 2/4 [00:00<00:00,  2.74it/s]

...

 75%|███████▌  | 3/4 [00:01<00:00,  1.77it/s]

700:	learn: 41.7918100	test: 113.1775974	best: 112.9082913 (598)	total: 691ms	remaining: 295ms
Stopped by overfitting detector  (120 iterations wait)

bestTest = 112.9082913
bestIteration = 598

Shrink model to first 599 iterations.
Learning rate set to 0.056075
0:	learn: 918.4413770	test: 1091.9479139	best: 1091.9479139 (0)	total: 1.1ms	remaining: 1.1s
100:	learn: 115.8336954	test: 155.1619172	best: 155.1619172 (100)	total: 94.3ms	remaining: 840ms
200:	learn: 85.7240143	test: 143.6074800	best: 143.5897433 (199)	total: 190ms	remaining: 755ms
300:	learn: 67.8355175	test: 140.3070148	best: 140.0562148 (283)	total: 284ms	remaining: 660ms
400:	learn: 57.8109922	test: 139.4355905	best: 139.0930068 (370)	total: 373ms	remaining: 557ms
500:	learn: 50.7195616	test: 139.1753009	best: 138.6516088 (468)	total: 467ms	remaining: 465ms
600:	learn: 45.5436043	test: 138.1171129	best: 137.9939750 (593)	total: 565ms	remaining: 375ms


100%|██████████| 4/4 [00:02<00:00,  1.67it/s]

700:	learn: 41.1923637	test: 138.4555881	best: 137.7725489 (616)	total: 659ms	remaining: 281ms
Stopped by overfitting detector  (120 iterations wait)

bestTest = 137.7725489
bestIteration = 616

Shrink model to first 617 iterations.
Learning rate set to 0.045739
0:	learn: 941.8583218	total: 1.07ms	remaining: 1.07s
1:	learn: 903.3356403	total: 2.23ms	remaining: 1.11s
2:	learn: 867.4338567	total: 3.48ms	remaining: 1.16s
3:	learn: 834.7435650	total: 4.66ms	remaining: 1.16s
4:	learn: 801.8728473	total: 5.84ms	remaining: 1.16s
5:	learn: 770.5099743	total: 7.18ms	remaining: 1.19s
6:	learn: 743.6092356	total: 8.33ms	remaining: 1.18s
7:	learn: 714.5435674	total: 9.42ms	remaining: 1.17s
8:	learn: 687.7845711	total: 10.6ms	remaining: 1.17s
9:	learn: 662.8326937	total: 11.8ms	remaining: 1.16s
10:	learn: 636.8086098	total: 13ms	remaining: 1.17s
11:	learn: 611.5992821	total: 14.4ms	remaining: 1.18s
12:	learn: 589.1453815	total: 15.6ms	remaining: 1.19s
13:	learn: 567.1183941	total: 16.9ms	remaining:

78:	learn: 144.6670463	total: 111ms	remaining: 1.29s
79:	learn: 143.7393999	total: 113ms	remaining: 1.3s
80:	learn: 142.8838159	total: 115ms	remaining: 1.3s
81:	learn: 142.1386919	total: 116ms	remaining: 1.3s
82:	learn: 140.9909267	total: 117ms	remaining: 1.29s
83:	learn: 140.3923328	total: 118ms	remaining: 1.29s
84:	learn: 139.1563157	total: 119ms	remaining: 1.28s
85:	learn: 137.9694267	total: 121ms	remaining: 1.28s
86:	learn: 137.2366790	total: 122ms	remaining: 1.28s
87:	learn: 136.6905128	total: 123ms	remaining: 1.27s
88:	learn: 135.9437285	total: 124ms	remaining: 1.27s
89:	learn: 135.3539241	total: 125ms	remaining: 1.27s
90:	learn: 134.5095131	total: 127ms	remaining: 1.26s
91:	learn: 133.9212260	total: 128ms	remaining: 1.26s
92:	learn: 133.4532234	total: 129ms	remaining: 1.26s
93:	learn: 132.9681822	total: 130ms	remaining: 1.26s
94:	learn: 132.2518461	total: 132ms	remaining: 1.25s
95:	learn: 131.5700922	total: 133ms	remaining: 1.25s
96:	learn: 130.9872589	total: 134ms	remaining: 1.

## 27

In [ ]:
catboost_evaluation(train_ft_27, test_ft_27)

  0%|          | 0/4 [00:00<?, ?it/s]

Learning rate set to 0.056075
0:	learn: 3279.8283282	test: 2751.7252323	best: 2751.7252323 (0)	total: 1.08ms	remaining: 1.08s
100:	learn: 363.9066220	test: 444.4461283	best: 444.4461283 (100)	total: 96.8ms	remaining: 862ms
200:	learn: 259.0960450	test: 375.7517684	best: 375.7517684 (200)	total: 195ms	remaining: 775ms
300:	learn: 205.0303301	test: 370.2185783	best: 370.2185783 (300)	total: 290ms	remaining: 672ms


 25%|██▌       | 1/4 [00:00<00:01,  2.07it/s]

400:	learn: 170.8312272	test: 375.0525443	best: 369.7272027 (310)	total: 385ms	remaining: 576ms
Stopped by overfitting detector  (120 iterations wait)

bestTest = 369.7272027
bestIteration = 310

Shrink model to first 311 iterations.
Learning rate set to 0.056075
0:	learn: 3282.4099573	test: 2685.2822371	best: 2685.2822371 (0)	total: 1.04ms	remaining: 1.04s
100:	learn: 357.7468711	test: 1394.9630115	best: 1279.7300202 (26)	total: 98.3ms	remaining: 875ms


 50%|█████     | 2/4 [00:00<00:00,  3.14it/s]

...

 75%|███████▌  | 3/4 [00:01<00:00,  1.97it/s]

600:	learn: 134.5667709	test: 400.8343000	best: 398.1658919 (556)	total: 579ms	remaining: 384ms
Stopped by overfitting detector  (120 iterations wait)

bestTest = 398.1658919
bestIteration = 556

Shrink model to first 557 iterations.
Learning rate set to 0.056075
0:	learn: 3208.4346653	test: 3614.9148959	best: 3614.9148959 (0)	total: 957us	remaining: 956ms
100:	learn: 363.5678372	test: 384.4456022	best: 384.4456022 (100)	total: 98.6ms	remaining: 877ms
200:	learn: 262.3511711	test: 329.5085130	best: 329.5085130 (200)	total: 191ms	remaining: 761ms
300:	learn: 205.4141621	test: 323.4290981	best: 321.8368119 (256)	total: 296ms	remaining: 688ms
400:	learn: 172.1686696	test: 320.7671953	best: 320.3503235 (362)	total: 390ms	remaining: 582ms
500:	learn: 149.5035534	test: 325.0745443	best: 319.8539751 (405)	total: 480ms	remaining: 478ms


100%|██████████| 4/4 [00:02<00:00,  1.99it/s]

Stopped by overfitting detector  (120 iterations wait)

bestTest = 319.8539751
bestIteration = 405

Shrink model to first 406 iterations.
Learning rate set to 0.045739
0:	learn: 3270.0275217	total: 1.04ms	remaining: 1.04s
1:	learn: 3145.0224191	total: 2.29ms	remaining: 1.14s
2:	learn: 3023.6124966	total: 3.56ms	remaining: 1.19s
3:	learn: 2906.1169363	total: 4.76ms	remaining: 1.19s
4:	learn: 2793.4387728	total: 5.92ms	remaining: 1.18s
5:	learn: 2694.7101593	total: 7.04ms	remaining: 1.17s
6:	learn: 2596.9579200	total: 8.15ms	remaining: 1.16s
7:	learn: 2495.9480091	total: 9.32ms	remaining: 1.16s
8:	learn: 2405.8307776	total: 10.6ms	remaining: 1.17s
9:	learn: 2315.0535533	total: 11.7ms	remaining: 1.16s
10:	learn: 2226.0042580	total: 12.9ms	remaining: 1.16s
11:	learn: 2145.1737123	total: 14.1ms	remaining: 1.16s
12:	learn: 2067.4672397	total: 15.3ms	remaining: 1.16s
13:	learn: 1992.6554719	total: 16.4ms	remaining: 1.16s
14:	learn: 1918.1961837	total: 17.6ms	remaining: 1.16s
15:	learn: 1851.7

119:	learn: 369.0928978	total: 144ms	remaining: 1.06s
120:	learn: 367.8045415	total: 146ms	remaining: 1.06s
121:	learn: 366.7428816	total: 148ms	remaining: 1.07s
122:	learn: 365.3579525	total: 149ms	remaining: 1.06s
123:	learn: 364.2319706	total: 150ms	remaining: 1.06s
124:	learn: 363.0390570	total: 152ms	remaining: 1.06s
125:	learn: 361.6014084	total: 153ms	remaining: 1.06s
126:	learn: 360.7951850	total: 154ms	remaining: 1.06s
127:	learn: 359.6557284	total: 155ms	remaining: 1.06s
128:	learn: 358.8804068	total: 156ms	remaining: 1.05s
129:	learn: 357.8751450	total: 157ms	remaining: 1.05s
130:	learn: 356.9112082	total: 158ms	remaining: 1.05s
131:	learn: 355.1393892	total: 160ms	remaining: 1.05s
132:	learn: 353.4865545	total: 161ms	remaining: 1.05s
133:	learn: 352.6910656	total: 162ms	remaining: 1.05s
134:	learn: 351.4434521	total: 163ms	remaining: 1.04s
135:	learn: 350.3053229	total: 164ms	remaining: 1.04s
136:	learn: 348.4615134	total: 165ms	remaining: 1.04s
137:	learn: 347.2582823	tota

## 28

In [ ]:
catboost_evaluation(train_ft_28, test_ft_28)

## 29

In [ ]:
catboost_evaluation(train_ft_29, test_ft_29)

## 30

In [ ]:
catboost_evaluation(train_ft_30, test_ft_30)

## 31

In [ ]:
catboost_evaluation(train_ft_31, test_ft_31)

## 32

In [ ]:
catboost_evaluation(train_ft_32, test_ft_32)

## 33

In [ ]:
catboost_evaluation(train_ft_33, test_ft_33)

## 34

In [ ]:
catboost_evaluation(train_ft_34, test_ft_34)

## 35

In [ ]:
catboost_evaluation(train_ft_35, test_ft_35)

## 36

In [ ]:
catboost_evaluation(train_ft_36, test_ft_36)

## 37

In [ ]:
catboost_evaluation(train_ft_37, test_ft_37)

## 38

In [ ]:
catboost_evaluation(train_ft_38, test_ft_38)

## 39

In [ ]:
catboost_evaluation(train_ft_39, test_ft_39)

## 40

In [ ]:
catboost_evaluation(train_ft_40, test_ft_40)

## 41

In [ ]:
catboost_evaluation(train_ft_41, test_ft_41)

## 42

In [ ]:
catboost_evaluation(train_ft_42, test_ft_42)

## 43

In [ ]:
catboost_evaluation(train_ft_43, test_ft_43)

## 44

In [ ]:
catboost_evaluation(train_ft_44, test_ft_44)

## 45

In [ ]:
train_ft_45.head()

In [ ]:
catboost_evaluation(train_ft_45, test_ft_45)

## 46

In [ ]:
catboost_evaluation(train_ft_46, test_ft_46)

## 47

In [ ]:
catboost_evaluation(train_ft_47, test_ft_47)

## 48

In [ ]:
catboost_evaluation(train_ft_48, test_ft_48)

  0%|          | 0/4 [00:00<?, ?it/s]

Learning rate set to 0.056075
0:	learn: 435.8082070	test: 426.0047489	best: 426.0047489 (0)	total: 7.76ms	remaining: 7.75s
100:	learn: 50.4696050	test: 51.4951093	best: 51.4951093 (100)	total: 338ms	remaining: 3.01s
200:	learn: 37.1645367	test: 48.4369455	best: 48.4369455 (200)	total: 527ms	remaining: 2.1s
300:	learn: 30.4730652	test: 47.2886433	best: 47.1406848 (278)	total: 786ms	remaining: 1.82s
400:	learn: 26.1409762	test: 46.2687603	best: 46.2673307 (383)	total: 1.06s	remaining: 1.58s
500:	learn: 23.1641415	test: 45.2188164	best: 45.2188164 (500)	total: 1.27s	remaining: 1.27s
600:	learn: 20.9259447	test: 44.5218772	best: 44.4767405 (587)	total: 1.46s	remaining: 968ms
700:	learn: 18.9924159	test: 44.3555614	best: 44.1725219 (682)	total: 1.67s	remaining: 711ms
800:	learn: 17.4589108	test: 44.2650239	best: 44.1725219 (682)	total: 1.83s	remaining: 455ms
Stopped by overfitting detector  (120 iterations wait)

bestTest = 44.17252186
bestIteration = 682

Shrink model to first 683 iteratio

 25%|██▌       | 1/4 [00:02<00:06,  2.02s/it]

Learning rate set to 0.056075
0:	learn: 437.7187580	test: 402.9508498	best: 402.9508498 (0)	total: 3.69ms	remaining: 3.69s
100:	learn: 49.1501790	test: 83.1354082	best: 71.6984238 (54)	total: 625ms	remaining: 5.56s


 50%|█████     | 2/4 [00:03<00:03,  1.53s/it]

...

 75%|███████▌  | 3/4 [00:07<00:02,  2.67s/it]

700:	learn: 18.2256565	test: 73.5672233	best: 73.2322756 (606)	total: 3.71s	remaining: 1.58s
Stopped by overfitting detector  (120 iterations wait)

bestTest = 73.23227563
bestIteration = 606

Shrink model to first 607 iterations.
Learning rate set to 0.056075
0:	learn: 431.1733452	test: 478.5949716	best: 478.5949716 (0)	total: 3.01ms	remaining: 3s
100:	learn: 49.5187983	test: 72.2624334	best: 71.2310212 (94)	total: 199ms	remaining: 1.77s
200:	learn: 36.6414707	test: 70.7492814	best: 70.5917602 (171)	total: 387ms	remaining: 1.54s
300:	learn: 29.8565041	test: 68.1319596	best: 68.1319596 (300)	total: 621ms	remaining: 1.44s
400:	learn: 25.6610740	test: 66.3508725	best: 66.3435228 (399)	total: 807ms	remaining: 1.21s
500:	learn: 22.6721883	test: 65.6830242	best: 65.6830242 (500)	total: 991ms	remaining: 987ms
600:	learn: 20.3918819	test: 64.7689811	best: 64.7689811 (600)	total: 1.1s	remaining: 729ms
700:	learn: 18.5608962	test: 64.5452461	best: 64.4298626 (692)	total: 1.19s	remaining: 509ms


100%|██████████| 4/4 [00:08<00:00,  2.22s/it]

999:	learn: 14.6753586	test: 63.4833103	best: 63.4063415 (959)	total: 1.47s	remaining: 0us

bestTest = 63.40634151
bestIteration = 959

Shrink model to first 960 iterations.
Learning rate set to 0.045739
0:	learn: 439.0210062	total: 1ms	remaining: 1s
1:	learn: 421.9922600	total: 2.21ms	remaining: 1.1s
2:	learn: 405.0690325	total: 3.47ms	remaining: 1.15s
3:	learn: 389.3634620	total: 4.75ms	remaining: 1.18s
4:	learn: 373.8947416	total: 6.05ms	remaining: 1.2s
5:	learn: 359.4563248	total: 7.21ms	remaining: 1.19s
6:	learn: 346.9854644	total: 8.84ms	remaining: 1.25s
7:	learn: 332.9834964	total: 10ms	remaining: 1.24s
8:	learn: 320.2793047	total: 11.2ms	remaining: 1.24s
9:	learn: 307.8258247	total: 12.4ms	remaining: 1.23s
10:	learn: 295.8364075	total: 13.6ms	remaining: 1.22s
11:	learn: 284.2489535	total: 14.8ms	remaining: 1.22s
12:	learn: 273.8613742	total: 15.9ms	remaining: 1.21s
13:	learn: 263.4935218	total: 17.1ms	remaining: 1.21s
14:	learn: 254.0819853	total: 18.4ms	remaining: 1.21s
15:	le

111:	learn: 52.3644656	total: 140ms	remaining: 1.11s
112:	learn: 52.1089158	total: 142ms	remaining: 1.12s
113:	learn: 51.7563533	total: 144ms	remaining: 1.12s
114:	learn: 51.5151745	total: 145ms	remaining: 1.12s
115:	learn: 51.2283825	total: 147ms	remaining: 1.12s
116:	learn: 50.9273010	total: 148ms	remaining: 1.12s
117:	learn: 50.7251195	total: 149ms	remaining: 1.11s
118:	learn: 50.4751509	total: 150ms	remaining: 1.11s
119:	learn: 50.2925568	total: 152ms	remaining: 1.11s
120:	learn: 50.0617451	total: 153ms	remaining: 1.11s
121:	learn: 49.9770896	total: 154ms	remaining: 1.11s
122:	learn: 49.7743868	total: 155ms	remaining: 1.1s
123:	learn: 49.5288113	total: 156ms	remaining: 1.1s
124:	learn: 49.3731900	total: 158ms	remaining: 1.1s
125:	learn: 49.2659688	total: 159ms	remaining: 1.1s
126:	learn: 49.1630416	total: 160ms	remaining: 1.1s
127:	learn: 49.0095676	total: 161ms	remaining: 1.1s
128:	learn: 48.7411940	total: 163ms	remaining: 1.1s
129:	learn: 48.5603986	total: 164ms	remaining: 1.09s


## 49

In [ ]:
catboost_evaluation(train_ft_49, test_ft_49)

## 50

In [ ]:
catboost_evaluation(train_ft_50, test_ft_50)

## 51

In [ ]:
catboost_evaluation(train_ft_51, test_ft_51)

## 52

In [ ]:
catboost_evaluation(train_ft_52, test_ft_52)

## 53

In [ ]:
catboost_evaluation(train_ft_53, test_ft_53)

## 54

In [ ]:
catboost_evaluation(train_ft_54, test_ft_54)

  0%|          | 0/4 [00:00<?, ?it/s]

Learning rate set to 0.056075
0:	learn: 1065.8261499	test: 1028.2819536	best: 1028.2819536 (0)	total: 2.58ms	remaining: 2.58s
100:	learn: 162.9441437	test: 269.9953934	best: 264.4033405 (65)	total: 338ms	remaining: 3.01s


 25%|██▌       | 1/4 [00:00<00:02,  1.05it/s]

Stopped by overfitting detector  (120 iterations wait)

bestTest = 264.4033405
bestIteration = 65

Shrink model to first 66 iterations.
Learning rate set to 0.056075
0:	learn: 1071.7644819	test: 969.2827724	best: 969.2827724 (0)	total: 5.75ms	remaining: 5.75s
100:	learn: 156.6473198	test: 609.6271949	best: 534.7429441 (28)	total: 643ms	remaining: 5.72s


 50%|█████     | 2/4 [00:01<00:01,  1.01it/s]

Stopped by overfitting detector  (120 iterations wait)

bestTest = 534.7429441
bestIteration = 28

Shrink model to first 29 iterations.
Learning rate set to 0.056075
0:	learn: 1046.2543141	test: 1242.1788887	best: 1242.1788887 (0)	total: 5.36ms	remaining: 5.35s
100:	learn: 160.4393375	test: 263.4711246	best: 262.8277070 (96)	total: 667ms	remaining: 5.94s
200:	learn: 122.6616173	test: 253.3858440	best: 253.2667820 (197)	total: 1.28s	remaining: 5.1s
300:	learn: 100.2163058	test: 249.6859165	best: 249.0809561 (275)	total: 1.87s	remaining: 4.35s


 75%|███████▌  | 3/4 [00:04<00:01,  1.73s/it]

Stopped by overfitting detector  (120 iterations wait)

bestTest = 249.0809561
bestIteration = 275

Shrink model to first 276 iterations.
Learning rate set to 0.056075
0:	learn: 1062.3857435	test: 1069.8573083	best: 1069.8573083 (0)	total: 2.76ms	remaining: 2.75s
100:	learn: 157.5452992	test: 269.3565174	best: 268.0327218 (95)	total: 331ms	remaining: 2.95s
200:	learn: 120.9473220	test: 265.5922380	best: 264.0522682 (150)	total: 582ms	remaining: 2.31s
Stopped by overfitting detector  (120 iterations wait)

bestTest = 264.0522682
bestIteration = 150

Shrink model to first 151 iterations.


100%|██████████| 4/4 [00:05<00:00,  1.35s/it]


Learning rate set to 0.045739
0:	learn: 1073.0681295	total: 2.71ms	remaining: 2.71s
1:	learn: 1029.7885001	total: 5.63ms	remaining: 2.81s
2:	learn: 988.7098793	total: 8.33ms	remaining: 2.77s
3:	learn: 950.9623664	total: 10.9ms	remaining: 2.73s
4:	learn: 912.5271656	total: 13.6ms	remaining: 2.71s
5:	learn: 876.1115458	total: 15.8ms	remaining: 2.61s
6:	learn: 842.9139912	total: 19.7ms	remaining: 2.8s
7:	learn: 809.0605880	total: 22.4ms	remaining: 2.77s
8:	learn: 778.4420596	total: 25ms	remaining: 2.76s
9:	learn: 749.3546508	total: 27.6ms	remaining: 2.73s
10:	learn: 722.8226532	total: 30.3ms	remaining: 2.73s
11:	learn: 696.1757055	total: 32.9ms	remaining: 2.71s
12:	learn: 672.0156412	total: 35.8ms	remaining: 2.71s
13:	learn: 646.7487862	total: 38.5ms	remaining: 2.71s
14:	learn: 623.5166138	total: 41.4ms	remaining: 2.72s
15:	learn: 601.4882485	total: 44.2ms	remaining: 2.72s
16:	learn: 580.4054771	total: 47ms	remaining: 2.72s
17:	learn: 560.7654728	total: 49.6ms	remaining: 2.71s
18:	learn: 

## 55

In [ ]:
catboost_evaluation(train_ft_55, test_ft_55)

## 56

In [ ]:
catboost_evaluation(train_ft_56, test_ft_56)

## 57

In [ ]:
catboost_evaluation(train_ft_57, test_ft_57)

## 58

In [ ]:
catboost_evaluation(train_ft_58, test_ft_58)

## 59

In [ ]:
catboost_evaluation(train_ft_59, test_ft_59)

## 60

In [ ]:
catboost_evaluation(train_ft_60, test_ft_60)

## 61

In [ ]:
catboost_evaluation(train_ft_61, test_ft_61)

## 62

In [ ]:
catboost_evaluation(train_ft_62, test_ft_62)

## 63

In [ ]:
catboost_evaluation(train_ft_63, test_ft_63)

## 64

In [ ]:
catboost_evaluation(train_ft_64, test_ft_64)

## 65

In [ ]:
catboost_evaluation(train_ft_65, test_ft_65)

## 66

In [ ]:
catboost_evaluation(train_ft_66, test_ft_66)

## 67

In [ ]:
catboost_evaluation(train_ft_67, test_ft_67)

## 68

In [ ]:
catboost_evaluation(train_ft_68, test_ft_68)

## 69

In [ ]:
catboost_evaluation(train_ft_69, test_ft_69)

## 70

In [ ]:
catboost_evaluation(train_ft_70, test_ft_70)

## 71

In [ ]:
catboost_evaluation(train_ft_71, test_ft_71)

  0%|          | 0/4 [00:00<?, ?it/s]

Learning rate set to 0.056075
0:	learn: 456.4375126	test: 410.3042990	best: 410.3042990 (0)	total: 1.09ms	remaining: 1.08s
100:	learn: 118.1212927	test: 116.1582824	best: 115.2698412 (74)	total: 95.7ms	remaining: 852ms
Stopped by overfitting detector  (120 iterations wait)

bestTest = 115.2698412
bestIteration = 74


 25%|██▌       | 1/4 [00:00<00:00,  4.16it/s]


Shrink model to first 75 iterations.
Learning rate set to 0.056075
0:	learn: 454.1406359	test: 430.6687311	best: 430.6687311 (0)	total: 1.05ms	remaining: 1.05s
100:	learn: 116.1876387	test: 252.4358833	best: 235.6915130 (47)	total: 93ms	remaining: 828ms


 50%|█████     | 2/4 [00:00<00:00,  4.48it/s]

...

 75%|███████▌  | 3/4 [00:00<00:00,  2.73it/s]

400:	learn: 59.1461470	test: 184.4088284	best: 183.7586399 (338)	total: 404ms	remaining: 604ms
Stopped by overfitting detector  (120 iterations wait)

bestTest = 183.7586399
bestIteration = 338

Shrink model to first 339 iterations.
Learning rate set to 0.056075
0:	learn: 450.3375462	test: 483.4408127	best: 483.4408127 (0)	total: 1.09ms	remaining: 1.09s


100%|██████████| 4/4 [00:01<00:00,  3.19it/s]

100:	learn: 113.5475325	test: 163.7895668	best: 162.1143526 (89)	total: 100ms	remaining: 894ms
200:	learn: 84.2556552	test: 168.7739405	best: 162.1143526 (89)	total: 199ms	remaining: 791ms
Stopped by overfitting detector  (120 iterations wait)

bestTest = 162.1143526
bestIteration = 89

Shrink model to first 90 iterations.
Learning rate set to 0.045739
0:	learn: 456.4197011	total: 1.08ms	remaining: 1.08s
1:	learn: 440.4600059	total: 2.37ms	remaining: 1.18s
2:	learn: 425.5271559	total: 3.59ms	remaining: 1.19s
3:	learn: 411.4240445	total: 4.94ms	remaining: 1.23s
4:	learn: 397.1798688	total: 6.21ms	remaining: 1.24s
5:	learn: 384.6799668	total: 7.39ms	remaining: 1.22s
6:	learn: 372.3185291	total: 8.56ms	remaining: 1.21s
7:	learn: 360.1236904	total: 9.82ms	remaining: 1.22s
8:	learn: 349.5684860	total: 11ms	remaining: 1.21s
9:	learn: 338.2865447	total: 12.3ms	remaining: 1.22s
10:	learn: 328.4799790	total: 13.7ms	remaining: 1.23s
11:	learn: 317.9396628	total: 14.9ms	remaining: 1.23s
12:	learn

30:	learn: 202.3225280	total: 41.7ms	remaining: 1.3s
31:	learn: 198.9879085	total: 42.7ms	remaining: 1.29s
32:	learn: 195.6748451	total: 43.7ms	remaining: 1.28s
33:	learn: 192.5811282	total: 44.7ms	remaining: 1.27s
34:	learn: 189.2130780	total: 45.8ms	remaining: 1.26s
35:	learn: 186.3562708	total: 46.9ms	remaining: 1.26s
36:	learn: 183.2133970	total: 48.3ms	remaining: 1.26s
37:	learn: 180.3674375	total: 49.5ms	remaining: 1.25s
38:	learn: 177.6722072	total: 50.8ms	remaining: 1.25s
39:	learn: 175.3143715	total: 52.2ms	remaining: 1.25s
40:	learn: 173.2075955	total: 53.5ms	remaining: 1.25s
41:	learn: 171.0622266	total: 54.7ms	remaining: 1.25s
42:	learn: 169.2280747	total: 55.9ms	remaining: 1.24s
43:	learn: 167.3840792	total: 57.1ms	remaining: 1.24s
44:	learn: 165.5203628	total: 58.4ms	remaining: 1.24s
45:	learn: 163.7928532	total: 59.7ms	remaining: 1.24s
46:	learn: 162.2052096	total: 60.9ms	remaining: 1.24s
47:	learn: 160.5815355	total: 62.1ms	remaining: 1.23s
48:	learn: 158.8623431	total:

## 72

In [ ]:
catboost_evaluation(train_ft_72, test_ft_72)

  0%|          | 0/4 [00:00<?, ?it/s]

Learning rate set to 0.056075
0:	learn: 185.7939329	test: 202.3624105	best: 202.3624105 (0)	total: 1ms	remaining: 1s
100:	learn: 34.0022975	test: 84.7204328	best: 77.7768460 (61)	total: 95.6ms	remaining: 851ms
Stopped by overfitting detector  (120 iterations wait)

bestTest = 77.77684596
bestIteration = 61

Shrink model to first 62 iterations.


 50%|█████     | 2/4 [00:00<00:00,  4.85it/s]

Learning rate set to 0.056075
0:	learn: 186.5817390	test: 187.3738199	best: 187.3738199 (0)	total: 1.12ms	remaining: 1.12s
100:	learn: 32.0966618	test: 137.6376299	best: 121.7107423 (33)	total: 92.4ms	remaining: 822ms
Stopped by overfitting detector  (120 iterations wait)

bestTest = 121.7107423
bestIteration = 33

Shrink model to first 34 iterations.
Learning rate set to 0.056075
0:	learn: 181.2066613	test: 248.1425482	best: 248.1425482 (0)	total: 964us	remaining: 964ms
100:	learn: 32.9514022	test: 121.9049770	best: 119.4953143 (68)	total: 92.3ms	remaining: 821ms
200:	learn: 24.5113852	test: 117.6016634	best: 117.6016634 (200)	total: 187ms	remaining: 745ms
300:	learn: 20.6894195	test: 115.0838629	best: 115.0260868 (294)	total: 287ms	remaining: 666ms
400:	learn: 18.1600433	test: 113.7521980	best: 113.7357660 (399)	total: 382ms	remaining: 570ms
500:	learn: 16.1372358	test: 113.1580362	best: 113.1580362 (500)	total: 494ms	remaining: 492ms
600:	learn: 14.4055017	test: 111.9740910	best: 11

 75%|███████▌  | 3/4 [00:01<00:00,  1.72it/s]

Stopped by overfitting detector  (120 iterations wait)

bestTest = 111.5079813
bestIteration = 816

Shrink model to first 817 iterations.
Learning rate set to 0.056075
0:	learn: 186.7229272	test: 189.1891100	best: 189.1891100 (0)	total: 955us	remaining: 954ms
100:	learn: 35.2936511	test: 73.3499898	best: 66.1142497 (45)	total: 93.2ms	remaining: 830ms


100%|██████████| 4/4 [00:01<00:00,  2.42it/s]

Stopped by overfitting detector  (120 iterations wait)

bestTest = 66.11424971
bestIteration = 45

Shrink model to first 46 iterations.
Learning rate set to 0.045739
0:	learn: 188.5734082	total: 990us	remaining: 990ms
1:	learn: 182.0171156	total: 2.24ms	remaining: 1.11s
2:	learn: 175.1666733	total: 3.58ms	remaining: 1.19s
3:	learn: 169.0000453	total: 4.79ms	remaining: 1.19s
4:	learn: 162.8241352	total: 5.99ms	remaining: 1.19s
5:	learn: 157.1348334	total: 7.23ms	remaining: 1.2s
6:	learn: 151.6568311	total: 8.39ms	remaining: 1.19s
7:	learn: 145.9855588	total: 9.55ms	remaining: 1.18s
8:	learn: 141.1164939	total: 10.8ms	remaining: 1.19s
9:	learn: 136.1495169	total: 12.1ms	remaining: 1.2s
10:	learn: 131.2440556	total: 13.3ms	remaining: 1.2s
11:	learn: 126.7594658	total: 14.5ms	remaining: 1.19s
12:	learn: 122.7091513	total: 15.9ms	remaining: 1.21s
13:	learn: 118.2968937	total: 17.1ms	remaining: 1.2s
14:	learn: 114.5679093	total: 18.5ms	remaining: 1.21s
15:	learn: 110.9420391	total: 19.9ms	re

115:	learn: 37.8426820	total: 146ms	remaining: 1.11s
116:	learn: 37.7165982	total: 147ms	remaining: 1.11s
117:	learn: 37.5963513	total: 150ms	remaining: 1.12s
118:	learn: 37.5085972	total: 152ms	remaining: 1.13s
119:	learn: 37.3666874	total: 153ms	remaining: 1.12s
120:	learn: 37.1022704	total: 154ms	remaining: 1.12s
121:	learn: 37.0202860	total: 156ms	remaining: 1.12s
122:	learn: 36.9641512	total: 157ms	remaining: 1.12s
123:	learn: 36.8476478	total: 158ms	remaining: 1.12s
124:	learn: 36.7856359	total: 159ms	remaining: 1.11s
125:	learn: 36.7316468	total: 161ms	remaining: 1.11s
126:	learn: 36.5781609	total: 162ms	remaining: 1.11s
127:	learn: 36.4841178	total: 163ms	remaining: 1.11s
128:	learn: 36.3882221	total: 164ms	remaining: 1.11s
129:	learn: 36.3587379	total: 165ms	remaining: 1.11s
130:	learn: 36.2455231	total: 167ms	remaining: 1.1s
131:	learn: 36.0827665	total: 168ms	remaining: 1.1s
132:	learn: 35.9573304	total: 170ms	remaining: 1.11s
133:	learn: 35.8636971	total: 172ms	remaining: 1

## 73

In [ ]:
catboost_evaluation(train_ft_73, test_ft_73)

## 74

In [ ]:
catboost_evaluation(train_ft_74, test_ft_74)

## 75

In [ ]:
catboost_evaluation(train_ft_75, test_ft_75)

## 76

In [ ]:
catboost_evaluation(train_ft_76, test_ft_76)

## 77

In [ ]:
catboost_evaluation(train_ft_77, test_ft_77)

## 78

In [ ]:
catboost_evaluation(train_ft_78, test_ft_78)

## 79

In [ ]:
catboost_evaluation(train_ft_79, test_ft_79)

## 80

In [ ]:
catboost_evaluation(train_ft_80, test_ft_80)

## 81

In [ ]:
catboost_evaluation(train_ft_81, test_ft_81)

## 82

In [ ]:
catboost_evaluation(train_ft_82, test_ft_82)

## 83

In [ ]:
catboost_evaluation(train_ft_83, test_ft_83)

## 84

In [ ]:
catboost_evaluation(train_ft_84, test_ft_84)

## 85

In [ ]:
catboost_evaluation(train_ft_85, test_ft_85)

## 86

In [ ]:
catboost_evaluation(train_ft_86, test_ft_86)

## 87

In [ ]:
catboost_evaluation(train_ft_87, test_ft_87)

## 88

In [ ]:
catboost_evaluation(train_ft_88, test_ft_88)

## 89

In [ ]:
catboost_evaluation(train_ft_89, test_ft_89)

## 90

In [ ]:
catboost_evaluation(train_ft_90, test_ft_90)

## 91

In [ ]:
catboost_evaluation(train_ft_91, test_ft_91)

## 92

In [ ]:
catboost_evaluation(train_ft_92, test_ft_92)

## 93

In [ ]:
catboost_evaluation(train_ft_93, test_ft_93)

## 94

In [ ]:
catboost_evaluation(train_ft_94, test_ft_94)

## 95

In [ ]:
catboost_evaluation(train_ft_95, test_ft_95)

## 96

In [ ]:
catboost_evaluation(train_ft_96, test_ft_96)

## 97

In [ ]:
catboost_evaluation(train_ft_97, test_ft_97)

## 98

In [ ]:
catboost_evaluation(train_ft_98, test_ft_98)

## 99

In [ ]:
catboost_evaluation(train_ft_99, test_ft_99)

## 100

In [ ]:
catboost_evaluation(train_ft_100, test_ft_100)

# 제출

In [ ]:
# submission = pd.read_csv(f"{DATA_PATH}sample_submission.csv")

In [ ]:
len(catboost_preds)

In [ ]:
train_ft.columns

In [ ]:
submission.iloc[:, 1] = catboost_preds

In [ ]:
submission.to_csv(f"{DATA_PATH}submission_cat_1.csv", index=False)

In [ ]:
submission.to_csv("submission_cat_1.csv", index=False)

In [ ]:
pd.DataFrame(smape_score_mean)

In [ ]:
train_ft_1.columns